# Notebook for CME Futures Challenge

# Downloading historical data for indices (S&P, NASDAQ, DJIA)

Imports

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import re
from typing import List, Dict

Make get_data function for downloading from yf

In [ ]:
timeframe = '1000mo' # set timeframe

def get_data(tickers: List):
    data_dictionary = {}
    for ticker in tickers:
        data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
    return data_dictionary

# Getting Data

We need continuized data for returns information about futures since they roll over at expiration dates. I bought some continuous data for cheap, it's not back adjusted, which we could do - but I may try doing CV folds on each contract instead.

In [ ]:
es_df = pd.read_csv('ES.txt') # E S&P
nq_df = pd.read_csv('NQ.txt') # E Nasdaq
fv_df = pd.read_csv('FV.txt') # 5yr us treasury note contracts
ty_df = pd.read_csv('TY.txt') # 10yr US treasury bond contracts
us_df = pd.read_csv('US.txt') # 30yr US treasury bond contracts
gc_df = pd.read_csv('GC.txt') # Gold contracts
cl_df = pd.read_csv('CL.txt') # Crude oil contracts
jy_df = pd.read_csv('JY.txt') # JPY/USD contracts
bp_df = pd.read_csv('BP.txt') # BP/USD contracts
eu_df = pd.read_csv('EU.txt') # EURO/USD contracts


Clean these up a bit

In [ ]:
def rename_price_columns(data, item):
    # Rename columns
    data.rename(columns={'Open': f'{item}_open', 'High': f'{item}_high', 'Low': f'{item}_low', 'Close': f'{item}_close', 'Volume': f'{item}_volume'}, inplace=True)

In [ ]:
contract_data = {
    'ES': es_df,
    'NQ': nq_df,
    'FV': fv_df,
    'TY': ty_df,
    'US': us_df, 
    'GC': gc_df,
    'CL': cl_df,
    'JY': jy_df,
    'BP': bp_df,
    'EU': eu_df
}

for contract, data in contract_data.items():
    # Make sure date is datetime and set as index
    try:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index(data['Date'], inplace=True)
        data.drop(columns=['Date'], inplace=True)
    except:
        print('Index already fixed.')

    # Rename columns
    rename_price_columns(data, contract)


In [ ]:
contract_data['ES']

References for later

In [ ]:
indexes = ['S&P', 'NASDAQ', 'DJIA']
contracts = contract_data.keys() # These are the ones we are training on

Get yahoo finance data

In [ ]:
tickers = ['^GSPC', '^IXIC', '^DJI'] # S&P, NASDAQ, DJIA

# Download data
data_dictionary = get_data(tickers)

# Turn to df
s_p = pd.DataFrame(data_dictionary['^GSPC'])
nasdaq = pd.DataFrame(data_dictionary['^IXIC'])
djia = pd.DataFrame(data_dictionary['^DJI'])

Fix structure

In [ ]:
# Flatten df so ticker doesn't span the top
s_p = s_p.droplevel(1, axis=1)
nasdaq = nasdaq.droplevel(1, axis=1)
djia = djia.droplevel(1, axis=1)

# Drop unnecessary columns
#s_p.drop(columns=['High', 'Low', 'Open'], inplace=True)
#nasdaq.drop(columns=['High', 'Low', 'Open'], inplace=True)
#djia.drop(columns=['High', 'Low', 'Open'], inplace=True)

index_data = {
    'S&P': s_p,
    'NASDAQ': nasdaq,
    'DJIA': djia,
}

for index, data in index_data.items():
    # Rename columns
    rename_price_columns(data, index)

# Downloading historical data for our factor model

We are going to model the index as a geometric brownian motion, with the mu factor being a linear regression model with numerous inputs.  

## Factor considerations:  
### <u>Term structure</u>
###### Term spread (10Y-3M)

### <u>Credit conditions</u>
###### IG spread (BAA-AAA)

### <u>Valuation</u>
###### Forward E/P - real 10Y
###### Dividend yield

### <u>Economic</u>
###### Fed funds
###### Inflation (CPI)
###### DXY change (dollar index)  

### Some of these we can get from yahoo finance:  

In [ ]:
tickers = [
    # Term structure
    '^TNX', # 10yr CBOE
    '^IRX', # 3m bill (on discount basis, need to convert to yield)

    # Economic
    'DX-Y.NYB', # Dollar index

    # Volatility
    '^VIX',

    # Gold could be a measure of macro activity?
    "GLD", # Gold ETF

    # Other metals
    "XME", # SPDR Metals and mining ETF

    # Credit
    'FLRN', # Bloomberg IG Credit ETF
    'JNK', # Bloomberg HY Credit ETF

]

data_dictionary = get_data(tickers)

for ticker, data in data_dictionary.items():
    data_dictionary[ticker] = pd.DataFrame(data_dictionary[ticker])
    data_dictionary[ticker] = data_dictionary[ticker].droplevel(1, axis=1)
    rename_price_columns(data_dictionary[ticker], ticker)

ten_yr = data_dictionary['^TNX']
three_m = data_dictionary['^IRX']
dollar_index = data_dictionary['DX-Y.NYB']
vix = data_dictionary['^VIX']

# Need to drop volume column here
ten_yr.drop(columns=['^TNX_volume'], inplace=True)
three_m.drop(columns=['^IRX_volume'], inplace=True)
dollar_index.drop(columns=['DX-Y.NYB_volume'], inplace=True)
vix.drop(columns=['^VIX_volume'], inplace=True)


#gold = data_dictionary['GLD']
#metals = data_dictionary['XME']
#igb = pd.DataFrame(data_dictionary['FLRN'])
#hyb = data_dictionary['JNK']

In [ ]:
yf_macros = [vix, ten_yr, three_m, dollar_index]

We should get dividend yield too

In [ ]:
div_data = {}
etfs = ['SPY', 'QQQ', 'DIA']

for etf in etfs:
    ticker = yf.Ticker(etf)
    div = ticker.dividends
    price = ticker.history(timeframe)['Close']

    # Calculate dividend yield
    div_12m = div.rolling(window='365D', min_periods=1).sum()
    div_12m = div_12m.reindex(price.index, method='ffill')
    div_yield = div_12m / price
    div_data[etf] = div_yield

Fix index for all 3 and rename columns

In [ ]:
div_data['SPY'].index = pd.to_datetime(div_data['SPY'].index).normalize().tz_localize(None) # Normalize puts date in format we want
div_data['QQQ'].index = pd.to_datetime(div_data['QQQ'].index).normalize().tz_localize(None) # Localize (none) makes sure it doesn't add our timezone
div_data['DIA'].index = pd.to_datetime(div_data['DIA'].index).normalize().tz_localize(None)

div_data['SPY'].name = 'S&P_div'
div_data['QQQ'].name = 'NASDAQ_div'
div_data['DIA'].name = 'DJIA_div'

### pandas_datareader lets us download fred data

In [14]:
from pandas_datareader import data as pdr
from datetime import datetime

In [ ]:
start = datetime(1990,1,1) # Start date for download

# Macroeconomic data
gdp = pdr.DataReader("GDP", "fred", start)
cpi = pdr.DataReader("CPIAUCSL", "fred", start)
fedfunds = pdr.DataReader("FEDFUNDS", "fred", start)
consumer_sentiment = pdr.DataReader("UMCSENT", "fred", start)
inflation_expectation = pdr.DataReader("MICH", "fred", start)

# For some reason this download doesn't have the most recent fed funds rate
fedfunds = pd.concat([fedfunds['FEDFUNDS'], pd.Series([4.08], index=[datetime(2025,9,17)])])

# Credit risk data
ig_spread = pdr.DataReader("BAMLC0A4CBBB", "fred", start)   # BofA BBB corp minus Treasuries
#hy_spread = pdr.DataReader("BAMLH0A0HYM2", "fred", start)   # BofA US High Yield spread
#baa_spread = pdr.DataReader("BAA10Y", "fred", start)        # Moody’s Baa – 10Y Treasury

In [ ]:
fred_data = [gdp, cpi, fedfunds, ig_spread, consumer_sentiment, inflation_expectation]

In [ ]:
# Last business day <= today
last_bday = pd.bdate_range(end=pd.Timestamp.today().normalize().tz_localize(None), periods=1)[0]

for i, df in enumerate(fred_data):
    s = df.squeeze() # make it a Series
    # Build a business-day index from the series start to last_bday
    bidx = pd.bdate_range(start=s.index.min(), end=last_bday)
    # Reindex to business days and forward-fill
    s = s.reindex(bidx, method='ffill')
    # Write back as a 1-col DataFrame with a proper name
    name = s.name if s.name else f"series_{i}"
    fred_data[i] = s.to_frame(name)

Let's build a master dataframe

In [ ]:
pd.set_option('display.max_columns', None)
data = s_p.join([nasdaq, djia])
data = data.join(contract for contract in contract_data.values())
data = data.join(macro for macro in yf_macros)
data = data.join(div for div in div_data.values())
data = data.join([fd for fd in fred_data])

Rename some columns

In [ ]:
data.rename(columns={'CPIAUCSL': 'CPI', 'BAMLC0A4CBBB': 'credit_spread', 'UMCSENT': 'consumer_sentiment', 'MICH': 'inflation_expectation', 'series_2': 'fed_funds'}, inplace=True)
data[data.index.year > 2000]

# Linear regression model

### Preprocessing Data

Let's check for NaNs (there's a ton)

In [ ]:
data.isna().sum()

Check data types

In [ ]:
data.dtypes

Fill forward for any gaps

In [ ]:
data = data.replace(0, np.nan)
data = data.fillna(method='ffill')

Drop others

In [ ]:
data = data.dropna()
data

# Feature Engineering

We need to be careful to not include things such as raw moving averages that will leak volatility information into our drift prediction  

Forward adjusting futures

In [ ]:
roll_rules = { # Rollover occurances: (months, days)
    'ES': (0,5),
    'NQ': (0,5),
    'FV': (1,1),
    'TY': (1,1),
    'US': (1,1),
    'GC': (1,2),
    'CL': (0,3),
    'JY': (0,2),
    'BP': (0,2),
    'EU': (0,2)
}

In [ ]:
IMM_QTR = {3, 6, 9, 12}
GOLD_MONTHS = {2, 4, 6, 8, 10, 12}

def _third_fridays_quarterly(start, end):
    d = pd.date_range(start, end, freq="WOM-3FRI")
    return d[d.month.isin(IMM_QTR)]

def _bmonth_end_quarterly(start, end):
    # Last business day of IMM months
    all_month_ends = pd.date_range(start.normalize(), end.normalize(), freq="BM")
    return all_month_ends[all_month_ends.month.isin(IMM_QTR)]

def _third_wednesdays_quarterly(start, end):
    # 3rd Wednesday of IMM months
    weds = pd.date_range(start, end, freq="WOM-3WED")
    return weds[weds.month.isin(IMM_QTR)]

def _gold_expiries(start, end):
    # 3rd-last business day of bi-monthly GC months (anchor only)
    mends = pd.date_range(start.normalize(), end.normalize(), freq="BM")
    mends = mends[mends.month.isin(GOLD_MONTHS)]
    # 3rd last BD in that month = BM - 2 BDays
    return (mends - pd.offsets.BusinessDay(2))

def _cl_expiries(start, end):
    # Crude "expiration" day (last trade day): 3 business days before the 25th of the PRIOR month
    # We'll generate it for EVERY contract month in range.
    cl_expiries = []
    # iterate month starts to month ends
    month_starts = pd.date_range(start.normalize(), end.normalize(), freq="MS")
    for ms in month_starts:
        # prior month 25th
        prior_month_25th = (ms - pd.offsets.MonthBegin(1)) + pd.offsets.Day(24)  # 25th of prior month
        # adjust 25th to previous business day if needed, then subtract 3 business days
        # (approximation: we ignore exchange holidays and use business days = weekdays)
        # Find the previous business day <= prior_month_25th
        prev_bd = prior_month_25th
        # step back to weekday if falls on weekend
        while prev_bd.weekday() >= 5:
            prev_bd -= pd.Timedelta(days=1)
        ltd = prev_bd - pd.offsets.BusinessDay(3)
        cl_expiries.append(ltd.normalize())
    return pd.DatetimeIndex(sorted(set(cl_expiries)))

def _expiry_calendar(symbol, start, end):
    s = symbol.upper()
    if s in {"ES", "NQ"}:
        return _third_fridays_quarterly(start, end)
    if s in {"FV", "TY", "US"}:
        return _bmonth_end_quarterly(start, end)
    if s == "GC":
        return _gold_expiries(start, end)
    if s == "CL":
        return _cl_expiries(start, end)
    if s in {"JY", "BP", "EU"}:
        return _third_wednesdays_quarterly(start, end)
    # Fallback: quarterly 3rd Fridays (reasonable default)
    return _third_fridays_quarterly(start, end)

# -----------------------
# main: compute rollover flags, episodes, and DTR
# -----------------------
def get_rollover_data(data: pd.DataFrame, item: str, roll_rules: dict):
    months_off, days_off = roll_rules[item]  # (months, days)

    # We add a buffer so we can see the next expiry at the end of the dataset
    start = pd.Timestamp(data.index.min()).normalize()
    end = (pd.Timestamp(data.index.max()).normalize() + pd.DateOffset(months=4))

    expiries = _expiry_calendar(item, start, end)

    # Compute roll dates per rule
    if months_off == 0:
        roll_dates = expiries - pd.offsets.BusinessDay(days_off)
    else:
        # For each expiry, go back N months to that month's LAST business day, then step back 'days_off'
        anchor_month_ends = (expiries - pd.offsets.MonthBegin(months_off))  # first of prior month(s)
        # move to that month's last business day
        anchor_bm_end = pd.to_datetime(anchor_month_ends).to_period('M').to_timestamp('M')
        # ensure it's a business day (BM gives last business day; if not, use BMonthEnd)
        anchor_bm_end = pd.DatetimeIndex(anchor_bm_end)  # calendar month-end
        # Convert to last BUSINESS day of that month
        anchor_last_bd = pd.DatetimeIndex([pd.Timestamp(d).to_period('M').to_timestamp('M') for d in anchor_bm_end])
        # Replace with BusinessMonthEnd for precision
        anchor_last_bd = pd.date_range(anchor_last_bd.min(), anchor_last_bd.max(), freq="BM").intersection(anchor_last_bd)
        # The above intersection collapses; safer approach:
        # Recompute last BD robustly per expiry:
        last_bds = []
        for e in expiries:
            # month = e - months_off
            mstart = (e - pd.offsets.MonthBegin(months_off))
            # last business day of that month:
            lbd = (mstart + pd.offsets.BMonthEnd(0))
            last_bds.append(lbd)
        anchor_last_bd = pd.DatetimeIndex(last_bds)

        roll_dates = anchor_last_bd - pd.offsets.BusinessDay(days_off)

    roll_dates = pd.DatetimeIndex(sorted(set(pd.to_datetime(roll_dates))))
    # Keep only roll dates that appear in our data index (typical)
    roll_dates = roll_dates.intersection(pd.DatetimeIndex(data.index))

    # Flags and episodes
    rflag = f"{item}_Rollover"
    eph = f"{item}_Episode"
    data[rflag] = 0
    if len(roll_dates):
        data.loc[roll_dates, rflag] = 1
    data[eph] = data[rflag].cumsum()

    # Days until next rollover (trading days)
    # Forward-fill the next roll date and count business days
    next_roll = (
        pd.Series(roll_dates, index=roll_dates)
        .reindex(data.index, method="bfill")
        .to_numpy()
    )
    days = np.full(len(data), np.nan)
    mask = ~pd.isna(next_roll)
    # convert to daily (no time) for busday_count
    left = data.index.to_numpy(dtype="datetime64[D]")
    right = pd.to_datetime(next_roll).to_numpy(dtype="datetime64[D]")
    days[mask] = np.busday_count(left[mask], right[mask])  # excludes the right endpoint
    data[f"{item}_Days_Until_Rollover"] = days

# -----------------------
# forward adjustment on roll days
# -----------------------
def rollover_forward_adjustment(data: pd.DataFrame, contract: str):
    rflag = f"{contract}_Rollover"
    close = f"{contract}_close"
    # Jump only on the roll day = today's close minus yesterday's close
    jump = data[rflag].fillna(0).astype(int) * (data[close] - data[close].shift(1).fillna(0.0))
    cum_adj = jump.cumsum()
    for col in ["open", "high", "low", "close"]:
        c = f"{contract}_{col}"
        if c in data.columns:
            data[c] = data[c] - cum_adj

In [ ]:
for contract in contracts:
    # Set info about expirations
    get_rollover_data(data, contract, roll_rules)

    # Forward adjust rollover
    rollover_forward_adjustment(data, contract)
data.dropna(inplace=True)

Function definitions to help out

In [ ]:
def rolling_mean(data, window):
    return data.rolling(window, min_periods=window).mean()

### Features

Technical Indicators

In [ ]:
def add_technical_indicators(data, items, cfg=None, dropna=True):
    if cfg is None:
        cfg = {
            "rsi_n": 14,
            "macd_fast": 12, "macd_slow": 26, "macd_signal": 9,
            "bb_n": 20, "bb_k": 2.0,
            "atr_n": 14
        }
    out = data.copy()

    for item in items:
        o = out[f"{item}_open"].astype(float)
        h = out[f"{item}_high"].astype(float)
        l = out[f"{item}_low"].astype(float)
        c = out[f"{item}_close"].astype(float)
        v = out[f"{item}_volume"].astype(float)

        # --- 1) RSI (Wilder smoothing) ---
        n = cfg["rsi_n"]
        delta = c.diff()
        gain = delta.clip(lower=0.0)
        loss = (-delta).clip(lower=0.0)
        avg_gain = gain.ewm(alpha=1/n, adjust=False, min_periods=n).mean()
        avg_loss = loss.ewm(alpha=1/n, adjust=False, min_periods=n).mean()
        rs = avg_gain / (avg_loss.replace(0, np.nan))
        rsi = 100 - (100 / (1 + rs))
        out[f"{item}_rsi_{n}"] = rsi

        # --- 2) MACD (EMA fast/slow, signal, hist) ---
        ema_fast = c.ewm(span=cfg["macd_fast"], adjust=False).mean()
        ema_slow = c.ewm(span=cfg["macd_slow"], adjust=False).mean()
        macd = ema_fast - ema_slow
        macd_signal = macd.ewm(span=cfg["macd_signal"], adjust=False).mean()
        macd_hist = macd - macd_signal
        out[f"{item}_macd"] = macd
        out[f"{item}_macd_signal"] = macd_signal
        out[f"{item}_macd_hist"] = macd_hist

        # --- 3) Bollinger Bands (+ %B, Bandwidth) ---
        bb_n = cfg["bb_n"]; bb_k = cfg["bb_k"]
        mavg = c.rolling(bb_n, min_periods=bb_n).mean()
        mstd = c.rolling(bb_n, min_periods=bb_n).std(ddof=0)
        upper = mavg + bb_k * mstd
        lower = mavg - bb_k * mstd
        pct_b = (c - lower) / (upper - lower)
        bandwidth = (upper - lower) / mavg
        out[f"{item}_bb_mean_{bb_n}"] = mavg
        out[f"{item}_bb_upper_{bb_n}_{int(bb_k)}"] = upper
        out[f"{item}_bb_lower_{bb_n}_{int(bb_k)}"] = lower
        out[f"{item}_bb_percent_b"] = pct_b
        out[f"{item}_bb_bandwidth"] = bandwidth

        # --- 4) ATR (Wilder, uses True Range) ---
        prev_c = c.shift(1)
        tr = np.maximum(h - l, np.maximum((h - prev_c).abs(), (l - prev_c).abs()))
        atr = tr.ewm(alpha=1/cfg["atr_n"], adjust=False, min_periods=cfg["atr_n"]).mean()
        out[f"{item}_atr_{cfg['atr_n']}"] = atr

        # --- 5) OBV (On-Balance Volume) ---
        sign = np.sign(c.diff().fillna(0.0))
        obv = (sign * v).fillna(0.0).cumsum()
        out[f"{item}_obv"] = obv

    if dropna:
        out = out.dropna()

    return out

In [ ]:
data = add_technical_indicators(data, contracts)

In [ ]:
def get_log_price(data, items):
    data = data.copy()
    
    ohlcv = ['open', 'high', 'low', 'close', 'volume']
    for item in items:
        for key in ohlcv:
            data[f'{item}_log_{key}'] = np.log(data[f'{item}_{key}'])
            data.drop(columns=[f'{item}_{key}'], inplace=True) # Drop originals

        data.dropna(inplace=True)
    return data

In [ ]:
def get_log_returns(data, items):
    data = data.copy()
    
    ohlcv = ['open', 'high', 'low', 'close', 'volume']
    for item in items:
        for key in ohlcv:
            data[f'{item}_log_{key}_ret'] = data[f'{item}_log_{key}'].diff()

        data[f'{item}_log_open_close_ret'] = data[f'{item}_log_close'] - data[f'{item}_log_open']

    data.dropna(inplace=True)
    return data

In [ ]:
# Get log transforms of price data
log_price_keys = indexes + list(contract_data.keys()) #+ ['GLD', 'XME', 'JNK']

# Log prices
data = get_log_price(data, log_price_keys)

# Log returns
data = get_log_returns(data, log_price_keys)

# Log for other price data (GDP)
data['log_GDP'] = np.log(data['GDP'])
data.drop(columns=['GDP'], inplace=True)
data.dropna(inplace=True)

Price/Volume Features

In [ ]:
def get_price_volume_features(data, items):
    data = data.copy()
    
    for item in items:
        # Price-based
        data[f'{item}_mom_1w'] = data[f'{item}_log_close'].diff(5) # Total price change / momentum indicator
        data[f'{item}_mom_3m'] = data[f'{item}_log_close'].diff(63)
        data[f'{item}_1m_rolling_price'] = rolling_mean(data[f'{item}_log_close'], 21)
        data[f'{item}_3m_rolling_price'] = rolling_mean(data[f'{item}_log_close'], 63)
        data[f'{item}_6m_rolling_price'] = rolling_mean(data[f'{item}_log_close'], 126)
        data[f'{item}_trend_speed_price'] = data[f'{item}_3m_rolling_price'].diff(5)  # How fast the 3m trend is changing on a weekly basis
        data[f'{item}_trend_dist_price'] = data[f'{item}_log_close'] - data[f'{item}_3m_rolling_price']

        # Volume-based (essentially the same as price for now)
        data[f'{item}_vlm_1w'] = data[f'{item}_log_volume'].diff(5) # Total volume change / momentum indicator
        data[f'{item}_vlm_1m'] = data[f'{item}_log_volume'].diff(21)
        data[f'{item}_vlm_3m'] = data[f'{item}_log_volume'].diff(63)
        data[f'{item}_3m_rolling_volume'] = rolling_mean(data[f'{item}_log_volume'], 63)
        data[f'{item}_trend_speed_volume'] = data[f'{item}_3m_rolling_volume'].diff(5)  # How fast the 3m trend is changing on a weekly basis
        data[f'{item}_trend_dist_volume'] = data[f'{item}_log_volume'] - data[f'{item}_3m_rolling_volume']

    return data

In [ ]:
data = get_price_volume_features(data, log_price_keys)
data.dropna(inplace=True)

Macro features

In [ ]:
data['log_GDP_ret'] = data['log_GDP'].diff()
data['log_GDP_ret_1y'] = data['log_GDP'].diff(252)
data['inflation_expectation_ret'] = data['inflation_expectation'].pct_change()
data['consumer_sentiment_ret'] = data['consumer_sentiment'].pct_change()
data['cpi_ret'] = data['CPI'].pct_change()
data.dropna(inplace=True)

Spreads

In [ ]:
data['Spread_NQ_ES_ret'] = data['ES_log_close_ret'] - data['NQ_log_close_ret']
data['Spread_NQ_ES_volume'] = data['ES_3m_rolling_volume'] - data['NQ_3m_rolling_volume']
data['Spread_ES_GC_ret'] = data['ES_log_close_ret'] - data['GC_log_close_ret']
data['Spread_ES_GC_volume'] = data['ES_3m_rolling_volume'] - data['GC_3m_rolling_volume']
data['Spread_FV_US_ret'] = data['FV_log_close_ret'] - data['US_log_close_ret']
data['Spread_FV_US_volume'] = data['FV_3m_rolling_volume'] - data['US_3m_rolling_volume']
data['Spread_TY_US_ret'] = data['TY_log_close_ret'] - data['US_log_close_ret']
data['Spread_TY_US_volume'] = data['TY_3m_rolling_volume'] - data['US_3m_rolling_volume']
data.dropna(inplace=True)

Calendar dummies

In [ ]:
try:
    month_dummies = pd.get_dummies(data.index.month, prefix="month", dtype=float)
    month_dummies.set_index(data.index, inplace=True)
    day_dummies = pd.get_dummies(data.index.dayofweek, prefix="weekday", dtype=float)
    day_dummies.set_index(data.index, inplace=True)
    presidency_year_dummies = pd.get_dummies(data.index.year % 4, prefix='pres_year', dtype='float')
    presidency_year_dummies.set_index(data.index, inplace=True)
    
    data = data.join([month_dummies, day_dummies, presidency_year_dummies])
except:
    print('Error: probably already created dummies.')

Volatility / Price Features (From ChatGPT)

In [ ]:
daily_vix_vol = data['^VIX_close'] / (100 * np.sqrt(252))
data['vix_implied_var'] = np.square(daily_vix_vol)
    # Measures the percentage change in the ^VIX over 1 week and 1 month
data['vix_mom_1w'] = data['^VIX_close'].pct_change(periods=5)
data['vix_mom_1m'] = data['^VIX_close'].pct_change(periods=21)

data[f'vix_trend_1m'] = rolling_mean(data[f'^VIX_close'], 21)
data[f'vix_trend_3m'] = rolling_mean(data[f'^VIX_close'], 63)
data[f'vix_trend_6m'] = rolling_mean(data[f'^VIX_close'], 126)

# Calculate how far the current VIX is from its trend (as a percentage)
data['vix_trend_dist'] = (data['^VIX_close'] - data['vix_trend_3m']) / data['vix_trend_3m']

In [ ]:
EPS = 1e-12

# If you already have a rolling_mean helper, keep it. Otherwise:
def rolling_mean(s, w):
    return s.rolling(w, min_periods=max(2, int(w*0.6))).mean()

def ewma_vol(r, lam=0.94):
    # EWMA variance per RiskMetrics: sigma_t^2 = (1-lam)*r_{t-1}^2 + lam*sigma_{t-1}^2
    # Use pandas ewm for convenience
    return r.pow(2).ewm(alpha=(1-lam), adjust=False).mean().clip(lower=0)

def rolling_autocorr(x, lag=1, window=63):
    # Rolling autocorrelation of x at a given lag
    # For stability, require at least ~60% of window
    minp = max(10, int(window*0.6))
    x0 = x
    x1 = x.shift(lag)
    return x0.rolling(window, min_periods=minp).corr(x1)

def realized_quarticity(r, window=63):
    # 3-month robust quarticity proxy (if daily): sum r^4 * (n / 3) approximation
    # Here we simply provide rolling sum of r^4; scaling optional depending on use
    minp = max(10, int(window*0.6))
    return (r.pow(4)).rolling(window, min_periods=minp).sum()

def build_vol_features(data, prefix, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94, target='close'):
    """
    Expects:
      data[f'{prefix}_log_price'] (daily log price)
      data[f'{prefix}_log_volume'] (daily log volume)
    Produces a suite of volatility-centric features for that prefix.
    """
    
    lp = data[f"{prefix}_log_{target}"]
    lv = data.get(f"{prefix}_log_volume", None)

    # Daily log return
    r = lp.diff()  # already log-price, so diff = log-return

    # --- Realized volatility proxies ---
    data[f"{prefix}_rv_1m"]  = r.rolling(day_w, min_periods=int(day_w*0.6)).var().clip(lower=0)          # variance
    data[f"{prefix}_rv_3m"]  = r.rolling(qtr_w, min_periods=int(qtr_w*0.6)).var().clip(lower=0)
    data[f"{prefix}_rv_1y"]  = r.rolling(yr_w,  min_periods=int(yr_w*0.6)).var().clip(lower=0)
    data[f"{prefix}_absrv_1m"] = r.abs().rolling(day_w, min_periods=int(day_w*0.6)).mean()               # mean |r|
    data[f"{prefix}_absrv_3m"] = r.abs().rolling(qtr_w, min_periods=int(qtr_w*0.6)).mean()

    # EWMA volatility (RiskMetrics-style)
    data[f"{prefix}_ewma_var"] = ewma_vol(r, lam=ewma_lambda)
    data[f"{prefix}_ewma_vol"] = np.sqrt(data[f"{prefix}_ewma_var"])

    # Volatility-of-volatility (how fast vol is changing)
    data[f"{prefix}_vol_speed_1w"] = data[f"{prefix}_rv_3m"].diff(5)                                      # weekly change in 3m var
    data[f"{prefix}_vol_mom_1m"]   = data[f"{prefix}_rv_3m"] - data[f"{prefix}_rv_1m"]                    # 3m vs 1m
    data[f"{prefix}_vol_mom_1y"]   = data[f"{prefix}_rv_1y"] - data[f"{prefix}_rv_3m"]

    # Volatility clustering proxies
    data[f"{prefix}_acf_sqret_lag1_3m"] = rolling_autocorr(r.pow(2), lag=1, window=qtr_w)
    data[f"{prefix}_acf_absret_lag1_3m"] = rolling_autocorr(r.abs(), lag=1, window=qtr_w)

    # Leverage effect proxy (contemporaneous corr between return and next day's vol)
    # Negative returns often precede higher vol; we proxy with corr(r_t, |r|_{t+1})
    data[f"{prefix}_lev_proxy_3m"] = r.shift(1).rolling(qtr_w, min_periods=int(qtr_w*0.6)).corr(r.abs()).shift(1)

    # Quarticity (heavy tails proxy)
    data[f"{prefix}_quarticity_3m"] = realized_quarticity(r, window=qtr_w)

    # Ratio features (normalized vol levels)
    data[f"{prefix}_vol_ratio_1m_3m"] = (data[f"{prefix}_rv_1m"] / (data[f"{prefix}_rv_3m"] + EPS))
    data[f"{prefix}_vol_ratio_3m_1y"] = (data[f"{prefix}_rv_3m"] / (data[f"{prefix}_rv_1y"] + EPS))
    data[f"{prefix}_ewma_over_3m"]    = (data[f"{prefix}_ewma_var"] / (data[f"{prefix}_rv_3m"] + EPS))

    # Price–volatility relation: distance from trend as a stress proxy
    data[f"{prefix}_price_trend_3m"]  = rolling_mean(lp, qtr_w)
    data[f"{prefix}_price_trend_dist"] = lp - data[f"{prefix}_price_trend_3m"]
    # Volatility when far below trend often spikes; include interaction
    data[f"{prefix}_vol_x_trend_dist"] = data[f"{prefix}_rv_1m"] * data[f"{prefix}_price_trend_dist"]

    # Volume–volatility links (if volume available)
    if lv is not None:
        dv = lv.diff()  # log-volume change
        data[f"{prefix}_vlm_var_1m"] = dv.rolling(day_w, min_periods=int(day_w*0.6)).var().clip(lower=0)
        data[f"{prefix}_vlm_var_3m"] = dv.rolling(qtr_w, min_periods=int(qtr_w*0.6)).var().clip(lower=0)
        # Corr between |r| and volume changes (vol–volume clustering)
        data[f"{prefix}_corr_absr_dlv_3m"] = r.abs().rolling(qtr_w, min_periods=int(qtr_w*0.6)).corr(dv)
        # Volume surprise proxy: current vs 3m trend
        data[f"{prefix}_vlm_trend_3m"] = rolling_mean(lv, qtr_w)
        data[f"{prefix}_vlm_trend_dist"] = lv - data[f"{prefix}_vlm_trend_3m"]
        # Vol reacts to volume surprises
        data[f"{prefix}_vol_x_vlm_surprise"] = data[f"{prefix}_rv_1m"] * data[f"{prefix}_vlm_trend_dist"]

    # 2. Calculate the spread against a realized variance feature you already have
    # We'll use a 1-month realized variance (rv_1m) for the S&P 500 as an example.
    # Ensure you have 'S&P_rv_1m' or a similar column from your previous code.
    data['{prefix}_vrp_spread'] = data['vix_implied_var'] - data[f'{prefix}_rv_1m']



    return data

# ---- Apply to all price series ---- #### both close and close_ret
for item in log_price_keys:
    data = build_vol_features(data, index, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94)
    data = build_vol_features(data, item, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94, target='close_ret')

# ---- Cross-index spillover features (optional but useful) ----
# Differences/spreads in contemporaneous vol across indices capture contagion/regime moves
data["SPX_minus_NDX_vol_1m"] = data["S&P_rv_1m"] - data["NASDAQ_rv_1m"]
data["SPX_minus_DJIA_vol_1m"] = data["S&P_rv_1m"] - data["DJIA_rv_1m"]
data["NDX_minus_DJIA_vol_1m"] = data["NASDAQ_rv_1m"] - data["DJIA_rv_1m"]
data['ES_minus_NQ_vol_1m'] = data["ES_rv_1m"] - data["NQ_rv_1m"]

In [ ]:
data.dropna(inplace=True)

### Split data

Training/testing 80/20 split

In [ ]:
import math

### Set target

In [ ]:
linear_model_target = 'next_ret'
drift = data.copy()

In [ ]:
for contract in contracts:
    drift[f'{contract}_{linear_model_target}'] = drift[f'{contract}_log_close_ret'].shift(-1)
drift.dropna(inplace=True)

In [ ]:
def split_data(data, split=0.8, embargo=2):
    cutoff = math.floor(len(data)*split)
    training_data = data.iloc[:(cutoff - embargo)]
    testing_data = data.iloc[(cutoff+embargo):]
    return training_data.copy(), testing_data.copy()

In [ ]:
drift_training_data, drift_testing_data = split_data(drift)

### Determine feature importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance

In [ ]:
def set_targets_features(target, training_data, contracts=contracts, additional_exclusions=[], additional_unscalables=[]):
    # Make sure we only fit on training_data and explanatory variables
    targets = [f'{contract}_{target}' for contract in contracts]
    month_dummies = [f'month_{month}' for month in range(1,13)]
    day_dummies = [f'weekday_{day}' for day in range(5)]
    presidency_year_dummies = day_dummies = [f'pres_year_{year}' for year in range(4)]
    rollover_dummy = [f'{contract}_Rollover' for contract in contracts]
    dummies = month_dummies + day_dummies + presidency_year_dummies + rollover_dummy
    additional_exclusions.extend([f'{contract}_Episode'])


    columns_to_exclude = [] + additional_exclusions

    unscalable_features = dummies + additional_unscalables
    scalable_features = [column for column in training_data.columns if column not in targets and column not in unscalable_features and column not in columns_to_exclude]
    all_features = unscalable_features + scalable_features

    return targets, all_features, unscalable_features, scalable_features

In [ ]:
drift_targets, drift_all_features, drift_unscale_features, drift_scale_features = set_targets_features(linear_model_target, drift_training_data)

Let's try using exp weight decay

In [ ]:
def determine_feature_importance(training_data, testing_data, features, target, contracts=contracts, mode='permutation', permutation_threshold = 0, selector_threshold='median'):
    pruned_features = {}

    for contract in contracts:
        y_train = training_data[f'{contract}_{target}']
        y_test = testing_data[f'{contract}_{target}']
        X_train = training_data[features]
        X_test = testing_data[features]

        # Fit random forest
        rf = RandomForestRegressor(
            n_estimators = 500,
            max_depth=None,
            random_state=42,
            n_jobs=-1
        )
        rf.fit(X_train, y_train)

        if mode == 'permutation':
            result = permutation_importance(
                rf, X_test, y_test, # Make sure on test data
                n_repeats = 30,
                random_state=42,
                n_jobs=-1,
            )

            importances = pd.Series(result.importances_mean, index=X_train.columns).sort_values(ascending=False)
            selected_features = importances[importances > permutation_threshold].index.tolist()

            pruned_features[contract] = {
                'train_reduced': X_train[selected_features],
                'test_reduced': X_test[selected_features],
                'selected_features': selected_features
            }


        if mode == 'random_forest':
            selector = SelectFromModel(rf, threshold=selector_threshold, prefit=True)
            selected_features = training_data[features].columns[selector.get_support()]

            pruned_features[contract] = {
                'train_reduced': selector.transform(X_train),
                'test_reduced': selector.transform(X_test),
                'selected_features': selected_features
            }

    return pruned_features

Pickling for saving compute expensive dicts

In [ ]:
import os, pickle, gzip, bz2, lzma
from pathlib import Path
from typing import Callable, Any, Tuple

In [ ]:
def _opener(path: Path, mode: str):
    ext = path.suffix.lower()
    if ext in ('.pkl', '.pickle'):
        return path.open(mode)
    if ext == '.gz':
        return gzip.open(path, mode)
    if ext == '.bz2':
        return bz2.open(path, mode)
    if ext in ('.xz', '.lzma'):
        return lzma.open(path, mode)
    raise ValueError(f"Unsupported extension: {ext}")

def get_or_build_results(
    path: str,
    build_fn: Callable[..., dict],
    *args, **kwargs
) -> Tuple[dict, str]:

    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)

    # 1) Try load
    try:
        with _opener(p, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:

        try:
            # 2) Build
            results = build_fn(*args, **kwargs)

            # 3) Save
            with _opener(p, 'wb') as f:
                pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print(f'Exception occured: {e}')

    return results

In [ ]:
# Random forest for now because it's faster
pruned_features = get_or_build_results('model_data/drift_features_rf.pkl.gz', determine_feature_importance, drift_training_data, drift_testing_data, drift_all_features, 'next_ret', mode='random_forest')

for contract in contracts:
    print(pruned_features[contract]['selected_features'])

### Normalize inputs

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [ ]:
def scale_features(training_data, testing_data, features):
    scaler = RobustScaler()
    scaler.fit(training_data[features]) # Fitting on training data

    train_scaled = training_data.copy()
    test_scaled = testing_data.copy()

    # Scale
    train_scaled[features] = scaler.transform(training_data[features])
    test_scaled[features] = scaler.transform(testing_data[features])

    # Save info on standardization for later
    #scaler_mu = pd.Series(scaler.mean_, index=features)
    #scaler_std = pd.Series(scaler.scale_, index=features)
    return train_scaled, test_scaled #, scaler_mu, scaler_std

In [ ]:
drift_train_scaled, drift_test_scaled = scale_features(drift_training_data, drift_testing_data, drift_scale_features)

### Linear Regression

In [ ]:
import warnings
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.decomposition import PCA
from sklearn.exceptions import ConvergenceWarning
from sklearn.cross_decomposition import PLSRegression
from mlxtend.evaluate import GroupTimeSeriesSplit

We are going to test with multiple different linear models to account for collinearity

In [ ]:
# Choose which models we want to use
linear_models = ['ridge', 'lasso', 'enet', 'pca'] 

In [ ]:
# Function to get print results from the models
def eval_and_report(y_true, y_pred, model_name):
    print(f"{model_name:18s} | R^2: {r2_score(y_true, y_pred):.4f} | RMSE: {root_mean_squared_error(y_true, y_pred):.6f}")

Training function

In [ ]:
def train_models(train_scaled, test_scaled, target, contracts=contracts, pruned_features=pruned_features, models=linear_models):
    # Suppress all convergence warnings
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    results = {}

    for contract in contracts:
        results[contract] = {}
        features = pruned_features[contract]['selected_features']

        print(f"\n=== Target: {f'{contract}_{target}'} ===")
        X_train = train_scaled[features].copy()
        y_train = train_scaled[f'{contract}_{target}'].copy()
        X_test = test_scaled[features].copy()
        y_test = test_scaled[f'{contract}_{target}'].copy()

        # Create time series splits that isolate each contract
        ep_train = train_scaled[f'{contract}_Episode']
        gts = GroupTimeSeriesSplit(n_splits = ep_train.nunique()-2, test_size=1, window_type='expanding', gap_size=1)
        
        tscv = list(gts.split(
            X = X_train,
            y = y_train,
            groups=ep_train.values
        ))

        # 1. Ordinary Least Squares (OLS)
        if 'ols' in models:
            ols = LinearRegression()
            ols.fit(X_train, y_train)
            yhat_ols = ols.predict(X_test)
            eval_and_report(y_test, yhat_ols, "OLS")

            # Print top coefficients
            ols_coef = pd.Series(ols.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top OLS coeffs:\n", ols_coef.head(10))

            results[contract].update({
                "ols_model": ols,
                "ols_coefs": ols_coef,
                "train_data_ols": pd.Series(ols.predict(X_train), index=y_train.index, name=f"ols_train"),
                "yhat_ols": pd.Series(yhat_ols, index=y_test.index, name=f"{contract}_{target}_ols_pred"),
            })

        # 2. Ridge with CV over alphas (time-series CV)
        if 'ridge' in models:
            alphas = np.logspace(-4, 6, 100)

            ridge = RidgeCV(alphas=alphas, cv=tscv, fit_intercept=True)
            ridge.fit(X_train, y_train)
            yhat_ridge = ridge.predict(X_test)
            eval_and_report(y_test, yhat_ridge, f"Ridge (alpha={ridge.alpha_:.4g})")

            # Print top coefficients
            ridge_coef = pd.Series(ridge.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top Ridge coeffs:\n", ridge_coef.head(10))
            
            results[contract].update({
                "ridge_model": ridge,
                "ridge_coefs": ridge_coef,
                "train_data_ridge": pd.Series(ridge.predict(X_train), index=y_train.index, name=f"ridge_train"),
                "yhat_ridge": pd.Series(yhat_ridge, index=y_test.index, name=f"{contract}_{target}_ridge_pred"),
            })

        # 3. Lasso with CV over alphas (time-series CV)
        if 'lasso' in models and not 'lasso_ridge' in models:
            alphas = np.logspace(-4, 6, 100)

            lasso = LassoCV(alphas=alphas, cv=tscv, fit_intercept=True, n_jobs=-1)
            lasso.fit(X_train, y_train)
            yhat_lasso = lasso.predict(X_test)
            eval_and_report(y_test, yhat_lasso, f"Lasso (alpha={lasso.alpha_:.4g})")

            # Print top coefficients
            lasso_coef = pd.Series(lasso.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top Lasso coeffs:\n", lasso_coef.head(10))

            results[contract].update({
                "lasso_model": lasso,
                "lasso_coefs": lasso_coef,
                "train_data_lasso": pd.Series(lasso.predict(X_train), index=y_train.index, name=f"lasso_train"),
                "yhat_lasso": pd.Series(yhat_lasso, index=y_test.index, name=f"{contract}_{target}_lasso_pred"),
            })

        # 4. ElasticNet with CV over alphas and l1_ratios (time-series CV)
        if 'enet' in models:
            alphas = np.logspace(-4, 6, 100)
            l1_ratios = np.arange(.1, 1, .1)   # 1.0 == Lasso, 0.0 == Ridge

            enet = ElasticNetCV(
                alphas=alphas,
                l1_ratio=l1_ratios,
                cv=tscv,
                fit_intercept=True,
                max_iter=20000,
                n_jobs = -1
            )
            enet.fit(X_train, y_train)

            yhat_enet = enet.predict(X_test)
            eval_and_report(y_test, yhat_enet, f"ElasticNet (alpha={enet.alpha_:.4g}, l1_ratio={enet.l1_ratio_})")

            # Print top coefficients
            enet_coef = pd.Series(enet.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top ElasticNet coeffs:\n", enet_coef.head(10))

            results[contract].update({
                "enet_model": enet,
                "enet_coegs": enet_coef,
                "train_data_enet": pd.Series(enet.predict(X_train), index=y_train.index, name=f"enet_train"),
                "yhat_enet": pd.Series(yhat_enet, index=y_test.index, name=f"{contract}_{target}_enet_pred"),
            })

        # 5. PCA on OLS
        if 'pca' in models:
            pca = PCA(n_components=.9).fit(X_train) # keep x% of variance and fit to training set
            train_pca = pca.transform(X_train)
            test_pca = pca.transform(X_test)

            ols_pca = LinearRegression()
            ols_pca.fit(train_pca, y_train)
            yhat_pca = ols_pca.predict(test_pca)
            eval_and_report(y_test, yhat_pca, "OLS+PCA")

            results[contract].update({
                "pca_model": ols_pca,
                "train_data_pca": pd.Series(ols_pca.predict(train_pca), index=y_train.index, name=f"pca_train"),
                "yhat_pca": pd.Series(yhat_pca, index=y_test.index, name=f"{contract}_{target}_pca_pred")
            })

        # 6. Use lasso into ridge
        if 'lasso_ridge' in models:
            alphas = np.logspace(-4, 6, 100)

            lasso = LassoCV(alphas=alphas, cv=tscv, fit_intercept=True)
            lasso.fit(X_train, y_train)
            yhat_lasso = lasso.predict(X_test)
            eval_and_report(y_test, yhat_lasso, f"Lasso (alpha={lasso.alpha_:.4g})")

            # Print top coefficients
            lasso_coef = pd.Series(lasso.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top Lasso coeffs:\n", lasso_coef.head(10))

            results[contract].update({
                "lasso_model": lasso,
                "lasso_coefs": lasso_coef,
                "train_data_lasso": pd.Series(lasso.predict(X_train), index=y_train.index, name=f"lasso_train"),
                "yhat_lasso": pd.Series(yhat_lasso, index=y_test.index, name=f"{contract}_{target}_lasso_pred"),
            })
        
            # Get columns where coefficients are nonzero
            important_features = X_train.columns[np.abs(lasso.coef_) > 0]
            X_train_reduced = X_train[important_features]
            X_test_reduced = X_test[important_features]

            alphas = np.logspace(-4, 3, 30)

            lasso_ridge = RidgeCV(alphas=alphas, cv=tscv, fit_intercept=True)
            lasso_ridge.fit(X_train_reduced, y_train)
            yhat_lasso_ridge = lasso_ridge.predict(X_test_reduced)
            eval_and_report(y_test, yhat_lasso_ridge, f"Lasso_Ridge (alpha={lasso_ridge.alpha_:.4g})")

            # Print top coefficients
            lasso_ridge_coef = pd.Series(lasso_ridge.coef_, index=important_features).sort_values(key=np.abs, ascending=False)
            print("Top Lasso_Ridge coeffs:\n", lasso_ridge_coef.head(10))

            results[contract].update({
                "lasso_ridge_model": lasso_ridge,
                "lasso_ridge_coefs": lasso_ridge_coef,
                "train_data_lasso_ridge": pd.Series(lasso_ridge.predict(X_train_reduced), index=y_train.index, name=f"lasso_ridge_train"),
                "yhat_lasso_ridge": pd.Series(yhat_lasso_ridge, index=y_test.index, name=f"{contract}_{target}_lasso_ridge_pred"),
            })

        # 7. Partial Least Squares (PLS) Regression
        if 'pls' in models:
            # First, find the best number of components using cross-validation
            best_n = -1
            best_score = -np.inf
            
            # Test a range of components (e.g., from 1 to 20)
            for n in range(1, 8):
                pls_cv = PLSRegression(n_components=n)
                # Use a time-series friendly CV
                scores = cross_val_score(pls_cv, X_train, y_train, cv=tscv, scoring='r2')
                
                if np.mean(scores) > best_score:
                    best_score = np.mean(scores)
                    best_n = n

            print(f"PLS best n_components: {best_n}")

            # Now, fit the final PLS model with the optimal number of components
            pls = PLSRegression(n_components=best_n)
            pls.fit(X_train, y_train)
            yhat_pls = pls.predict(X_test)
            eval_and_report(y_test, yhat_pls, f"PLS (n={best_n})")

            # can get coefficients if needed, but they are in PLS component space
            # pls_coef = pd.Series(pls.coef_, index=features)

            # Store results
            results[contract].update({
                "pls_model": pls,
                # "pls_coefs": pls_coef,
                "train_data_pls": pd.Series(pls.predict(X_train).ravel(), index=y_train.index, name=f"pls_train"),
                "yhat_pls": pd.Series(yhat_pls.ravel(), index=y_test.index, name=f"{contract}_{target}_pls_pred"),
            })
            


    return results

In [ ]:
drift_results = get_or_build_results('model_data/drift_training_results.pkl.gz', train_models, drift_train_scaled, drift_test_scaled, linear_model_target, models=linear_models)

Make into new df

In [ ]:
def df_from_predictions(data, target, results, contracts=contracts, models=linear_models):
    df = data.copy()

    for contract in contracts:
        for model in models:
            df[f'{contract}_{target}_pred_{model}'] = pd.concat([results[contract][f'train_data_{model}'],results[contract][f'yhat_{model}']])
    df.dropna(inplace=True)

    return df


In [ ]:
drift = df_from_predictions(drift, linear_model_target, drift_results)

In [ ]:
drift

Plot these results

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def plot_results(data, baseline, measures, title, subtitles, contracts=contracts, models=linear_models):
    fig = make_subplots(
        rows=len(contracts), cols=1, shared_xaxes=True, vertical_spacing=0.06,
        subplot_titles=[contract for contract in contracts]
    )

    for i, (contract) in enumerate(contracts, start=1):
        show_leg = (i == 1)

        # True next_ret
        fig.add_trace(
            go.Scatter(x=data.index, y=data[f'{contract}_{baseline}'], name=f'{contract}_{baseline}',
                    mode="lines", line=dict(width=1.6),
                    showlegend=show_leg, legendgroup="true"),
            row=i, col=1
        )

        # Plot predictions
        for model in models:
            for measure in measures:
                fig.add_trace(
                    go.Scatter(x=data.index, y=data[f'{contract}_{measure}_{model}'], name=f"{model} prediction",
                            mode="lines", line=dict(width=1.4, dash="dot"),
                            showlegend=show_leg, legendgroup="ridge"),
                    row=i, col=1
                )

    fig.update_layout(
        title=title,
        height=2500,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(t=80, r=30, b=80, l=70),
        legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="left", x=0)
    )

    for r in range(1, len(contracts) + 1):
        fig.update_yaxes(title_text=subtitles, row=r, col=1)

    fig.show()

In [ ]:
plot_results(drift, 'log_close_ret', [f'{linear_model_target}_pred'], 'Linear Regression on Drift', 'Log Return')

Overall returns

In [ ]:
def build_price_path(df, pred, linear_model_target=linear_model_target, contracts=contracts, models=linear_models):
    df = df.copy()

    for contract in contracts:
        for model in models:
            df[f'{contract}_{pred}_pred_{model}'] = np.exp(df[f'{contract}_log_close'].iloc[0]) * np.exp(df[f'{contract}_{linear_model_target}_pred_{model}'].cumsum())

            df[f'{contract}_close'] = np.exp(df[f'{contract}_log_close'].iloc[0]) * np.exp(df[f'{contract}_log_close_ret'].cumsum())

    return df

In [ ]:
plot_drift = build_price_path(drift, 'price')
plot_results(plot_drift, 'close', ['price_pred'], 'Price Path Evolution of Drift', 'Price')

# Volatility Modeling

In [ ]:
vol_data = data.copy()
vol_data


In [ ]:
# Squared residuals from previous -- using ElasticNet
# Turns out we actually need to also take the log because they are so tiny

linear_drift_model = 'enet'
volatility_model_target = 'log_squared_residual'

for contract in contracts:
    vol_data[f'{contract}_{volatility_model_target}'] = np.log(np.square(drift[f'{contract}_{linear_model_target}'] - drift[f'{contract}_{linear_model_target}_pred_{linear_drift_model}']))
vol_data.dropna(inplace=True)

In [ ]:
vol_training_data, vol_testing_data = split_data(vol_data)
vol_targets, vol_all_features, vol_unscale_features, vol_scale_features = set_targets_features(volatility_model_target, vol_training_data)


# Random forest for now because it's faster
vol_pruned_features = get_or_build_results('model_data/vol_features.pkl.gz', determine_feature_importance, vol_training_data, vol_testing_data, vol_all_features, volatility_model_target, mode='random_forest')

for contract in contracts:
    print(vol_pruned_features[contract]['selected_features'])

In [ ]:
vol_train_scaled, vol_test_scaled = scale_features(vol_training_data, vol_testing_data, vol_scale_features)

In [ ]:
vol_test_scaled.dropna(inplace=True)

In [ ]:
volatility_models = ['ridge', 'lasso', 'pca']

In [ ]:
vol_results = get_or_build_results('model_data/vol_results.pkl.gz', train_models, vol_train_scaled, vol_test_scaled, volatility_model_target, pruned_features=vol_pruned_features, models=volatility_models)

Make into new df

In [ ]:
diffusion = df_from_predictions(vol_data, volatility_model_target, vol_results, models=volatility_models)

In [ ]:
plot_results(diffusion, volatility_model_target, [f'{volatility_model_target}_pred'], 'Linear Regression on Diffusion', 'Log Residuals', models=volatility_models)

Convert to variance / std

In [ ]:
for contract in contracts:
    for model in volatility_models:
        diffusion[f'{contract}_variance_pred_{model}'] = np.exp(diffusion[f'{contract}_{volatility_model_target}_pred_{model}'])
        diffusion[f'{contract}_std_pred_{model}'] = np.sqrt(diffusion[f'{contract}_variance_pred_{model}'])

In [ ]:
predictions = pd.merge(drift, diffusion, left_index=True, right_index=True, how='inner', suffixes=('', '_to_drop'))
predictions = predictions.drop(columns=[col for col in predictions.columns if col.endswith('_to_drop')])

Let's add a rolling IC column to test with damping mu projections later

In [ ]:
for contract in contracts:
    predictions[f'{contract}_{linear_model_target}'] = predictions[f'{contract}_log_close'].diff().shift(-1).dropna(inplace=True)
    for model in linear_models:
        predictions[f'{contract}_IC_{model}'] = predictions[f'{contract}_{linear_model_target}'].rolling(window=21).corr(predictions[f'{contract}_{linear_model_target}_pred_{model}'])

## Looks like we obtained a pretty smooth path. There are many extreme values not being captured -- Thus the motivation for GARCH

# GARCH Volatility Model

In [ ]:
from arch import arch_model

Set up some GARCH models, check their fit our data

In [ ]:
garch_data = drift.copy()
garch_scale = 10


# Get drift residuals:
for contract in contracts:
    garch_data[f'{contract}_drift_residuals_{linear_drift_model}'] = drift[f'{contract}_{linear_model_target}'] - drift[f'{contract}_{linear_model_target}_pred_{linear_drift_model}'] # Calculate residual from enet
    garch_data[f'{contract}_drift_residuals_{linear_drift_model}'] = garch_data[f'{contract}_drift_residuals_{linear_drift_model}'] * garch_scale # Scaling for better convergence

# Split data
garch_training, garch_testing = split_data(garch_data)

# Models, distributions, and paramters to check

garch_models = {
    'Garch': 'Garch',
    'Egarch': 'Egarch',
    'GJR_Garch': 'Garch',
    'APARCH': 'APARCH'
}

dists = ['normal', 't', 'skewt']

params = {
    'Garch': {'p': 1, 'o': 0, 'q': 1},
    'Egarch': {'p': 2, 'o': 1, 'q': 1},
    'GJR_Garch': {'p': 1, 'o': 1, 'q': 1},
    'APARCH': {'p': 1, 'o': 1, 'q': 1}
}

Check the fit to our data

In [ ]:
for contract in contracts: # Test against normal / t
    print(f'=== {contract} ===')
    for model in garch_models:
        for dist in dists: 
            
            try:
                garch = arch_model(garch_training[f'{contract}_drift_residuals_{linear_drift_model}'], vol=garch_models[model], **params[model], dist=dist) # Set model type
                garch_fit = garch.fit(disp='off') # Fit model, disp = off turn off output

                print(garch_fit.summary()) # Print results       
            except:
                print('\n')
                print(f'--- {model} with {dist} failed to fit for {contract} ---')
            print('\n')


Results: skew-t always prevails trailed closely by t, Egarch and GJR-Garch are very similar, looks like APARCH wins overall

Let's try a slightly more rigorous approach by grid searching.

In [ ]:
from itertools import product

In [ ]:
def qlike(y2, fvar):
    y2 = np.asarray(y2, float)
    fvar = np.asarray(fvar, float)
    return np.mean(y2 / fvar + np.log(fvar))

def fit_one(y, vol_name, model_name, p, o, q, dist, mean='Constant', lags=0):
    kw = dict(mean=mean, lags=lags, vol=vol_name, dist=dist, p=p, q=q)
    if model_name != 'Garch':  # only asym metrics use 'o'
        kw['o'] = o
    am = arch_model(y, **kw)
    return am.fit(disp='off')

def fit_arx(y, X, vol_name, model_name, p, o, q, dist, mean, lags):
    kw = dict(mean=mean, lags=lags, x=X, vol=vol_name, dist=dist, p=p, q=q)
    if model_name != 'Garch':  # only asym metrics use 'o'
        kw['o'] = o
    am = arch_model(y, **kw)
    return am.fit(disp='off')

def rolling_oos_qlike(y, vol_name, model_name, p, o, q, dist,
                      mean='Constant', lags=0, refit_every=21, start_frac=0.6):
    """1-step-ahead variance forecasts with periodic refits."""
    y = y.dropna()
    n = len(y)
    start = int(n * start_frac)
    fvars = []
    res = None
    for t in range(start, n - 1):
        if res is None or ((t - start) % refit_every == 0):
            res = fit_one(y.iloc[:t + 1], vol_name, model_name, p, o, q, dist, mean, lags)
        fvar = res.forecast(horizon=1, reindex=False).variance.values[-1, 0]
        fvars.append(fvar)
    return qlike((y**2).iloc[start+1:], fvars)

def garch_grid_search(
    garch_training,             # DataFrame with your series, e.g., f'{contract}_drift_residuals_enet'             # dict like {'Garch':'GARCH','Egarch':'EGARCH','GJR_Garch':'GJR-GARCH','APARCH':'APARCH'}
    dists=('t','skewt'),
    P=(1,2,3), Q=(1,2,3), O=(0,1),
    means=('Constant', 'Zero'), lags=(0,1,2),
    contracts=contracts,
    garch_models = garch_models,
    compute_oos=False, refit_every=21, start_frac=0.6, arx=False, X=None
):
    rows = []
    for contract in contracts:
        y = garch_training[f'{contract}_drift_residuals_{linear_drift_model}'].dropna()
        for model_name, vol_name in garch_models.items():
            # Build parameter grid; for plain GARCH, force o=0 only
            grid_O = (0,) if model_name == 'Garch' else O
            for p, q, o, dist, lag, mean in product(P, Q, grid_O, dists, lags, means):
                try:
                    if arx:
                        res = fit_arx(y, X, vol_name, model_name, p, o, q, dist, mean, lag)
                    else:
                        res = fit_one(y, vol_name, model_name, p, o, q, dist, mean, lag)
                    aic = res.aic
                    bic = res.bic
                    llf = res.loglikelihood
                    row = {
                        'contract': contract,
                        'model': model_name,
                        'vol': vol_name,
                        'dist': dist,
                        'p': p, 'o': o, 'q': q,
                        'aic': aic, 'bic': bic, 'llf': llf,
                        'lags': lag,
                        'mean': mean
                    }
                    if compute_oos:
                        try:
                            oos = rolling_oos_qlike(y, vol_name, model_name, p, o, q, dist,
                                                    mean, lags, refit_every, start_frac)
                        except Exception:
                            oos = np.nan
                        row['oos_qlike'] = oos
                    rows.append(row)
                except Exception:
                    # skip failed fits
                    print('Failed fit, skipping')
                    continue
    out = pd.DataFrame(rows)
    if out.empty:
        return out
    sort_cols = ['oos_qlike','aic'] if compute_oos else ['aic']
    ascending = [True] * len(sort_cols)
    return out.sort_values(sort_cols, ascending=ascending).reset_index(drop=True)

In [ ]:
#results = get_or_build_results('model_data/garch_grid_search', garch_grid_search, garch_training)


In [ ]:
#results.drop_duplicates(subset=['contract'])

In [ ]:
garch_test_models = ['Egarch', 'APARCH']

## Forecast our testing period with the chosen models

In [ ]:
refit_freq = 21 # Frequency to refit at -- slightly larger helps with signal:noise, but not too big

for contract in contracts:
    for model in garch_test_models:
        drift_residuals = garch_data[f'{contract}_drift_residuals_{linear_drift_model}']

        scaled_predictions = []
        for i in range(len(garch_training), len(garch_training) + len(garch_testing)): # Walk forward, forecast each step and refit model based on new inputs

            # Ensure we fit on first day and at our frequency
            is_refit_day = (i - len(garch_training)) % refit_freq == 0
            
            if is_refit_day:
                garch = arch_model(drift_residuals.iloc[:i], vol=garch_models[model], **params[model], mean='Zero', dist='skewt')
                garch_fit = garch.fit(disp='off') # fit

            forecast = garch_fit.forecast(horizon=1)

            pred_variance = forecast.variance.iloc[-1].values[0]
            scaled_predictions.append(pred_variance)

        # Get exponentiation of std
        if model == 'APARCH':
            delta = garch_fit.params['delta']
        else:
            delta = 2
        
        unscaled_predictions = pd.Series(scaled_predictions, index=garch_testing.index) / (garch_scale ** delta) # Unscale (inputs scaled by 100 for convergence, var is squared, but APARCH may return delta =/= 2)
        final_predictions = unscaled_predictions ** (1 / delta)

        garch_testing[f'{contract}_std_pred_{model}'] = final_predictions

In [ ]:
garch_testing

In [ ]:
for contract in contracts:
    for model in garch_test_models:
        predictions[f'{contract}_std_pred_{model}'] = garch_testing[f'{contract}_std_pred_{model}']

# Backtesting

In [ ]:
from scipy.stats import norm # For confidence scaling testing

Sizing strategy 1: Using Merton portfolio optimization

In [ ]:
def kelly_fraction(mu, rf, sigma, fraction=.5, alpha=0.0, D=1.0):
    size = fraction * (mu - rf) / sigma**2 
    scale = (1 - alpha * D) / (1 - alpha) # Optional drawdown floor, alpha is min portfolio value, D is high / current wealth
    return size * scale

Sizing strategy 2: Using vol target

In [ ]:
def get_vol_target_sizing(target, vol):
    size = (target/vol)
    return size

Returns info

In [ ]:
def sharpe_and_mdd(df, col_equity, col_rf="ten_yr", periods_per_year=252):
    eq = df[col_equity]
    rf = df[col_rf]

    # compute portfolio simple returns
    port_rets = eq.pct_change().dropna()
    # align rf
    rf_aligned = rf.reindex(port_rets.index).astype(float)

    # excess returns (assuming rf is already per-period, e.g. daily)
    excess = port_rets - ((1+rf_aligned/100)**(1/periods_per_year)-1)
    sharpe = (excess.mean() / excess.std()) * np.sqrt(periods_per_year) if excess.std() > 0 else np.nan

    # max drawdown
    running_max = eq.cummax()
    drawdown = eq / running_max - 1.0
    max_dd = drawdown.min()

    return sharpe, max_dd

## Trading strategy 1: Trade based on our models

### Backtest

In [ ]:
bt_models = ['ridge', 'lasso', 'APARCH', 'Egarch'] # Volatility models

Get historical margin amounts

In [ ]:
historical_margins = {}

for contract in contracts:
    historical_margins[contract] = pd.read_csv(f'{contract}_Historical_Margin.csv')
    historical_margins[contract]['Date'] = pd.to_datetime(historical_margins[contract]['Date'])
    historical_margins[contract].set_index(historical_margins[contract]['Date'], inplace=True)
    historical_margins[contract].drop(columns=['Date'], inplace=True)
    historical_margins[contract].rename(columns={'Long Margin': f'{contract}_Long', 'Short Margin': f'{contract}_Short'}, inplace=True)

### Individual Contract BTs

Backtesting with margin functionality (with rules per CME future challenge)  
Note that I am omitting the minimum 10 per day contract trading rule because we can just buy and sell quickly to nullify the effect  
I am also omitting the liquidation before expiry rule because there are no expiries during the competition  

In [ ]:
# Create backtest df copy
bt = predictions.copy()
bt = bt.join([margin_data for margin_data in historical_margins.values()])
bt = bt.replace(0, pd.NA)
bt = bt.fillna(method='ffill')
bt_train, bt_test = split_data(bt)
linear_drift_model = 'enet'

# Set initial conditions
starting_cash = 500_000
commission = 2.5
drawdown_lock = -.2

initial_margin = 1.1 # 10% extra assumption on initial entry
tick_size = {
    'ES': 0.25, 
    'NQ': 0.25, 
    'FV': 0.0078125, 
    'TY': 0.015625, 
    'US': 0.03125, 
    'GC': 0.10, 
    'CL': 0.01, 
    'JY': 0.0000005,
    'BP': 0.0001, 
    'EU': 0.00005
}

tick_value = {
    'ES': 12.50, 
    'NQ': 5.00, 
    'FV': 7.8125, 
    'TY': 15.625, 
    'US': 31.25, 
    'GC': 10.00, 
    'CL': 10.00, 
    'JY': 6.25,
    'BP': 6.25, 
    'EU': 6.25
}

bt_data = {}

for contract in contracts:
    bt_data[contract] = {}

    for model in bt_models:
        bt_test[f'{contract}_cash_{model}'] = 0.0
        bt_test[f'{contract}_margin_{model}'] = 0.0
        bt_test.loc[bt_test.index[0], f'{contract}_cash_{model}'] = starting_cash
        bt_test[f'{contract}_signal_{model}'] = 0
        bt_test[f'{contract}_baseline_wealth'] = np.exp(bt_test[f'{contract}_log_close_ret'].cumsum()) * starting_cash
        bt_test[f'{contract}_margin_usage_{model}'] = 0


        bt_data[contract][model] = {
            'last_margin': 0,
            'last_cash': starting_cash,
            'position': 0,
            'average_vol_weight': 0,
            'average_confidence': 0,
            'average_vol': 0,
            'average_mu': 0,
            'average_rf': 0,
            'average_ret': 0,
            'high_water': 0,
        }

# Starts at second day
for idx, row in bt_test.iloc[1:].iterrows():
    for contract in contracts:
        vol_data = []
        for model in bt_models:
            last_margin = bt_data[contract][model]['last_margin']
            last_cash = bt_data[contract][model]['last_cash']

            multiplier = tick_value[contract] / tick_size[contract]
            position_size = bt_data[contract][model]['position']
            lock_up = False

            
            vol = row[f'{contract}_std_pred_{model}']
            vol_data.append(vol)
            vol_data = vol_data[-21:]

            if model not in garch_models.keys():
                mu = np.exp(row[f'{contract}_{linear_model_target}_pred_{linear_drift_model}'] - .5*vol**2) - 1 # I believe we need the vol adjustment for drift based on the way we did it for the regular models
            else:
                mu = np.exp(row[f'{contract}_{linear_model_target}_pred_{linear_drift_model}']) - 1
                
            rf = (1+row['^TNX_close']/100)**(1/252)-1 # Daily
            investment = 0
            ret = np.exp(row[f'{contract}_log_close_ret']) - 1
            delta = ret*np.exp(row[f'{contract}_log_open']) # This calculation is basically assuming we are making our trading decisions at the end of every day
            pred = mu*np.exp(row[f'{contract}_log_close'])
            

            # Update margin value
            pnl_today = bt_data[contract][model]['position'] * delta * multiplier

            # Calculate total equity
            last_margin += pnl_today
            total_equity = last_margin + last_cash

            # Calculate max portfolio value for drawdown constraints
            bt_data[contract][model]['high_water'] = max(total_equity, bt_data[contract][model]['high_water'])

            # Check maintenance requirements
            if position_size > 0:
                direction = 'Long'
                side = 1
            else:
                direction = 'Short'
                side = -1

            margin_difference = row[f'{contract}_{direction}']*position_size - last_margin
            if margin_difference > 0 and position_size != 0 :
                # If enough cash to top up, do it automatically
                if total_equity >= margin_difference:
                    last_cash -= margin_difference
                    last_margin += margin_difference
                else:
                    # Otherwise liquidate
                    bt_data[contract][model]['position'] = 0
                    last_cash += last_margin # Ignoring commission
                    last_margin = 0

            # Recalculate total equity
            total_equity = last_margin + last_cash

            # If drawdown criteria met, lock portfolio for day
            if ret <= drawdown_lock:
                lock_up = True

            if not lock_up:
                # Get bet sizing with fractional kelly -- default is .5
                leverage = 1 # May want to look into making this dynamic... tried a little bit with mixed results
                target_notional = kelly_fraction(mu, rf, vol, fraction=.5) * total_equity * leverage # Daily rf
                target_notional = max(-total_equity * leverage, min(target_notional, total_equity * leverage)) # Make sure we don't exceed equity

                # Get bet sizing with fixed vol target
                #base_weight = get_vol_target_sizing(.15 / np.sqrt(252), vol)
                #confidence = (mu - rf) / vol
                #k=2
                #target_weight = base_weight * norm.cdf(confidence*np.sqrt(252))*k # Annualize
                #target_notional = target_weight * total_equity
                #target_notional = max(-total_equity, min(target_notional, total_equity)) # Make sure we don't exceed equity / effective equity cap

                # Data logging
                bt_data[contract][model]['average_rf'] += rf
                bt_data[contract][model]['average_ret'] += ret
                bt_data[contract][model]['average_mu'] += mu
                bt_data[contract][model]['average_vol'] += vol
                #bt_data[contract][model]['average_vol_weight'] += base_weight
                #bt_data[contract][model]['average_confidence'] += norm.cdf(confidence*np.sqrt(252))*k

                # Execute trade / Update investment amount to reach target position
                if target_notional > 0:
                    direction = 'Long'
                else: 
                    direction = 'Short'

                notional_per_contract = np.exp(row[f'{contract}_log_close']) * multiplier
                initial_cost = row[f'{contract}_{direction}']*initial_margin # Get current maintenance * initial margin percentage requirement
                position = math.floor(target_notional/initial_cost) * .15 # Get number of contracts to calculate margin investment, ignoring commission
                investment = position*initial_cost - last_margin
                investment = max(-last_cash, min(investment, last_cash))

                bt_data[contract][model]['position'] = position

            else:
                investment = 0

            # Update cash
            last_cash = bt_data[contract][model]['last_cash'] = last_cash - investment
            bt_test.loc[idx, f'{contract}_cash_{model}'] = last_cash

            # Store portfolio value as last value for calculation
            bt_data[contract][model]['last_margin'] = last_margin + investment
            bt_test.loc[idx, f'{contract}_margin_{model}'] = bt_data[contract][model]['last_margin']
            bt_test.loc[idx, f'{contract}_margin_usage_{model}'] = last_margin/total_equity

            # Set signal for plotting later
            if investment > 0:
                bt_test.loc[idx, f'{contract}_signal_{model}'] = 1
            elif investment < 0:
                bt_test.loc[idx, f'{contract}_signal_{model}'] = -1
    
for contract in contracts:
    print(f'=== {contract} ===')
    total_value = f'{contract}_baseline_wealth'
    
    final_training_value = bt_test[total_value].iloc[-1]
    training_cagr = 100 * ((final_training_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
    print(f'Final value for {contract}, baseline: {final_training_value:.2f}')
    
    test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
    print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {training_cagr:.2f}%\n")
    for model in bt_models:
        
        total_value = f'{contract}_total_value_{model}'
        bt_test[total_value] = bt_test[f'{contract}_margin_{model}'] + bt_test[f'{contract}_cash_{model}']

        final_testing_value = bt_test[total_value].iloc[-1]
        testing_cagr = 100*((final_testing_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
        print(f'Final value for {contract}, {model}: {final_testing_value:.2f}')
        
        test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
        print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {testing_cagr:.2f}%")


        bt_data[contract][model]['average_mu'] /= len(bt_test)
        bt_data[contract][model]['average_vol'] /= len(bt_test)
        bt_data[contract][model]['average_rf'] /= len(bt_test)
        bt_data[contract][model]['average_ret'] /= len(bt_test)
        average_margin_util = bt_test[f'{contract}_margin_usage_{model}'].sum() / len(bt_test)
        print(f"Average Vol: {bt_data[contract][model]['average_vol']:.4f}, Average mu: {bt_data[contract][model]['average_mu']:.4f}, Average ret: {bt_data[contract][model]['average_ret']:.4f}")
        print(f"Average Margin Utilizization: {average_margin_util*100:.2f}%")
        print('\n')

In [ ]:
plot_results(bt_test, 'baseline_wealth', ['total_value'], 'Backtest Performance', 'Wealth', models=bt_models, contracts=contracts)

Might be slower, but much easier to implement strategy using iterative process

In [ ]:
# Create backtest df copy
bt = predictions.copy()
bt_training, bt_test = split_data(bt)

# Set initial conditions
starting_cash = 10000
bt_data = {}

for contract in contracts:
    bt_data[contract] = {}

    for model in bt_models:
        bt_test[f'{contract}_portfolio_value_{model}'] = 0.0
        bt_test[f'{contract}_cash_{model}'] = 0.0
        bt_test.loc[bt_test.index[0], f'{contract}_cash_{model}'] = starting_cash
        bt_test[f'{contract}_signal_{model}'] = 0
        bt_test[f'{contract}_baseline_wealth'] = np.exp(bt_test[f'{contract}_log_close_ret'].cumsum()) * 10000


        bt_data[contract][model] = {
            'last_value': 0,
            'last_cash': starting_cash,
            'average_vol_weight': 0,
            'average_confidence': 0,
            'average_vol': 0,
            'average_mu': 0,
            'average_rf': 0,
            'average_ret': 0
        }

# Starts at second day
for idx, row in bt_test.iloc[1:].iterrows():
    for contract in contracts:
        for model in bt_models:
            last_value = bt_data[contract][model]['last_value']
            last_cash = bt_data[contract][model]['last_cash']

            mu = np.exp(row[f'{contract}_{linear_model_target}_pred_{linear_drift_model}']) - 1
            vol = row[f'{contract}_std_pred_{model}']
            rf = (1+row['^TNX_close']/100)**(1/252)-1 # Daily
            investment = 0
            ret = np.exp(row[f'{contract}_log_close_ret'])

            # Update portfolio value
            current_value = last_value * ret

            # Calculate total equity
            total_equity = current_value + last_cash 

            # Get bet sizing with Merton
            #target_pos = get_merton_size(mu, rf, vol) * total_equity # Daily rf
            #target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity

            # Get bet sizing with vol target
            #target_pos = get_vol_target_sizing(.1/np.sqrt(252), vol) * total_equity
            #target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity
            
            # Hybrid - Scale it with signed confidence from mu
            base_weight = get_vol_target_sizing(.15 / np.sqrt(252), vol)
            confidence = (mu - rf) / vol # Essentially a sharpe ratio
            k=2
            target_weight = base_weight * norm.cdf(confidence*np.sqrt(252))*k # Annualize
            target_pos = target_weight * total_equity
            target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity

            # Data logging
            bt_data[contract][model]['average_rf'] += rf
            bt_data[contract][model]['average_ret'] += ret - 1
            bt_data[contract][model]['average_mu'] += mu
            bt_data[contract][model]['average_vol'] += vol
            bt_data[contract][model]['average_vol_weight'] += base_weight
            bt_data[contract][model]['average_confidence'] += norm.cdf(confidence*np.sqrt(252))*k

            # Bet sizing based on size of potential increase/decrease, signed
            #target_pos = (mu - rf)/abs(mu - rf) * total_equity
            #target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity

            # Execute trade / Update investment amount to reach target position
            investment = target_pos - current_value

            # Update cash
            last_cash = bt_data[contract][model]['last_cash'] = last_cash - investment
            bt_test.loc[idx, f'{contract}_cash_{model}'] = last_cash

            # Store portfolio value as last value for calculation
            bt_data[contract][model]['last_value'] = current_value + investment
            bt_test.loc[idx, f'{contract}_portfolio_value_{model}'] = bt_data[contract][model]['last_value']

            # Set signal for plotting later
            if investment > 0:
                bt_test.loc[idx, f'{contract}_signal_{model}'] = 1
            elif investment < 0:
                bt_test.loc[idx, f'{contract}_signal_{model}'] = -1
    
for contract in contracts:
    print(f'=== {contract} ===')
    total_value = f'{contract}_baseline_wealth'
    
    final_training_value = bt_test[total_value].iloc[-1]
    training_cagr = 100 * ((final_training_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
    print(f'Final value for {contract}, baseline: {final_training_value:.2f}')
    
    test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
    print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {training_cagr:.2f}%\n")
    for model in bt_models:
        
        total_value = f'{contract}_total_value_{model}'
        bt_test[total_value] = bt_test[f'{contract}_portfolio_value_{model}'] + bt_test[f'{contract}_cash_{model}']

        final_testing_value = bt_test[total_value].iloc[-1]
        testing_cagr = 100*((final_testing_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
        print(f'Final value for {contract}, {model}: {final_testing_value:.2f}')
        
        test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
        print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {testing_cagr:.2f}%")



        bt_data[contract][model]['average_vol_weight'] /= len(bt_test)
        bt_data[contract][model]['average_confidence'] /= len(bt_test)
        bt_data[contract][model]['average_mu'] /= len(bt_test)
        bt_data[contract][model]['average_vol'] /= len(bt_test)
        bt_data[contract][model]['average_rf'] /= len(bt_test)
        bt_data[contract][model]['average_ret'] /= len(bt_test)
        print(f"Average Vol: {bt_data[contract][model]['average_vol']:.4f}, Average mu: {bt_data[contract][model]['average_mu']:.4f}, Average ret: {bt_data[contract][model]['average_ret']:.4f}, Average rf: {bt_data[contract][model]['average_rf']:.4f}")
        print(f"Average Vol Weight: {bt_data[contract][model]['average_vol_weight']:.4f}, Average Confidence Scaling: {bt_data[contract][model]['average_confidence']:.4f}\n")
    print('\n')

In [ ]:
plot_results(bt_test, 'baseline_wealth', ['total_value'], 'Backtest Performance', 'Wealth', models=bt_models)

Trading strategy 2: Trading using bollinger bands and moving average

# Boll band construction

In [ ]:
# Boll bands with enet
def create_boll_bands(measure='std_pred', std=2, window=20):
    for index in indexes:
        for model in models:
            vol = predictions[f'{index}_{measure}_{model}']
            predictions[f'{index}_sma'] = predictions[f'{index}_Close'].rolling(window=window, min_periods=window).mean()
            predictions[f'{index}_upper_boll_{model}'] = std * vol * predictions[f'{index}_sma'] + predictions[f'{index}_sma']
            predictions[f'{index}_lower_boll_{model}'] = -std * vol * predictions[f'{index}_sma'] + predictions[f'{index}_sma']

In [ ]:
for index in indexes:
    for model in models:
        predictions[f'{index}_realized_vol_{model}'] = predictions[f'{index}_price_pred_{model}'].rolling(window=5,min_periods=5).std()

create_boll_bands()
plot_results(predictions, 'Close', ['upper_boll', 'lower_boll', 'price_pred'], 'Bollinger Bands', 'Price', indexes, ['enet'])

Let's see some stats on boll bands here

In [ ]:
def set_boll_counts():
    for index in indexes:
        px_col  = f'{index}_Close'
        sma_col = f'{index}_sma'

        for model in models:
            std_col   = f'{index}_std_pred_{model}'
            up_col    = f'{index}_upper_boll_{model}'
            low_col   = f'{index}_lower_boll_{model}'
            sig_col   = f'{index}_boll_signal_{model}'            # -1 lower break, +1 upper break, 0 inside
            touchU    = f'{index}_touch_upper_{model}'            # boolean
            touchL    = f'{index}_touch_lower_{model}'
            crossU    = f'{index}_cross_above_upper_{model}'      # boolean: crossed today
            crossL    = f'{index}_cross_below_lower_{model}'
            countAny  = f'{index}_cum_band_breaks_{model}'        # cumulative count
            countU    = f'{index}_cum_upper_breaks_{model}'
            countL    = f'{index}_cum_lower_breaks_{model}'
            inBand    = f'{index}_inside_band_{model}'
            widthCol  = f'{index}_band_width_{model}'             # relative width

            # 3) Booleans: touches (price outside band)
            price = predictions[px_col]
            upper = predictions[up_col]
            lower = predictions[low_col]

            predictions[touchU] = (price >= upper)
            predictions[touchL] = (price <= lower)
            predictions[inBand] = (~predictions[touchU] & ~predictions[touchL])

            # 4) True "cross" events (crossed today vs yesterday)
            prev_price = price.shift(1)
            prev_up    = upper.shift(1)
            prev_low   = lower.shift(1)

            # Cross above upper: was <= upper yesterday and > upper today
            predictions[crossU] = (prev_price <= prev_up) & (price > upper)
            # Cross below lower: was >= lower yesterday and < lower today
            predictions[crossL] = (prev_price >= prev_low) & (price < lower)

            # 5) Compact signal: +1 if price above upper, -1 if below lower, else 0
            predictions[sig_col] = np.select(
                [predictions[touchU], predictions[touchL]],
                [1, -1],
                default=0
            ).astype(int)

            # 6) Cumulative counts you can tally quickly
            predictions[countU]   = predictions[crossU].cumsum()
            predictions[countL]   = predictions[crossL].cumsum()
            predictions[countAny] = (predictions[crossU] | predictions[crossL]).cumsum()

            # 7) Optional: rolling 20-day counts if you want “recent frequency”
            predictions[f'{index}_roll20_breaks_{model}'] = (
                (predictions[crossU] | predictions[crossL]).rolling(20, min_periods=1).sum()
            )

            # 8) Band width (relative, helpful for diagnostics/screening)
            predictions[widthCol] = (upper - lower) / predictions[sma_col]

In [ ]:
import numpy as np
import pandas as pd

def forward_returns(price, horizons=(1,3,5,10)):
    """Calculates forward log returns for given horizons."""
    out = {}
    lp = np.log(price.astype(float))
    for h in horizons:
        out[h] = (lp.shift(-h) - lp)      # r_{t→t+h}, aligned at t
    return pd.DataFrame(out, index=price.index)

def reenter_band_within(pred_df, idx, mdl, horizon=5):
    """Boolean: did price re-enter band within 'horizon' days after being outside?"""
    px     = pred_df[f"{idx}_Close"]
    up     = pred_df[f"{idx}_upper_boll_{mdl}"]
    low    = pred_df[f"{idx}_lower_boll_{mdl}"]
    outside = (px > up) | (px < low)
    inside  = ~(outside)
    # rolling forward "any inside" within next N days (including next day)
    # Build a forward-looking window using shift(-k). We’ll OR across 1..N.
    any_inside_nextN = pd.Series(False, index=pred_df.index)
    for k in range(1, horizon+1):
        any_inside_nextN = any_inside_nextN | inside.shift(-k)
    return outside & any_inside_nextN

def summarize_boll_stats(predictions, indexes, models, horizons=(1,3,5,10), reenter_N=5):
    """
    Calculates various statistics for Bollinger Band signals, including hit rates,
    mean returns, and average win/loss amounts.
    """
    rows = []
    for idx in indexes:
        px = predictions[f"{idx}_Close"]
        fwd = forward_returns(px, horizons)
        for mdl in models:
            # Columns built earlier
            up     = predictions[f"{idx}_upper_boll_{mdl}"]
            low    = predictions[f"{idx}_lower_boll_{mdl}"]
            touchU = predictions[f"{idx}_touch_upper_{mdl}"].astype(bool)
            touchL = predictions[f"{idx}_touch_lower_{mdl}"].astype(bool)
            crossU = predictions[f"{idx}_cross_above_upper_{mdl}"].astype(bool)
            crossL = predictions[f"{idx}_cross_below_lower_{mdl}"].astype(bool)
            inside = predictions[f"{idx}_inside_band_{mdl}"].astype(bool)
            width  = (up - low) / predictions[f"{idx}_sma"]

            n = len(px.dropna())
            # base rates
            pct_inside = inside.mean()
            pct_touchU = touchU.mean()
            pct_touchL = touchL.mean()
            pct_crossU = crossU.mean()
            pct_crossL = crossL.mean()

            # --- NEW: Define more complex signals ---
            # Bearish: Touched upper band yesterday, and today it's back inside.
            mean_revert_U = touchU.shift(1) & inside
            # Bullish: Touched lower band yesterday, and today it's back inside.
            mean_revert_L = touchL.shift(1) & inside
            # --- END NEW ---

            # forward return frames aligned to signal dates
            sigs = {
                "touchU": touchU,
                "touchL": touchL,
                "crossU": crossU,
                "crossL": crossL,
                "mean_revert_U": mean_revert_U, # NEW
                "mean_revert_L": mean_revert_L, # NEW
            }

            # re-entry (mean reversion back inside band) within N days after being outside
            reenter = reenter_band_within(predictions, idx, mdl, horizon=reenter_N)
            pct_reenter_after_outside = reenter.mean()

            row = {
                "index": idx,
                "model": mdl,
                "obs": n,
                "% inside": pct_inside,
                "% touchU": pct_touchU,
                "% touchL": pct_touchL,
                "% crossU": pct_crossU,
                "% crossL": pct_crossL,
                "band_width_med%": float(np.nanmedian(width))*100.0,
                "band_width_p90%": float(np.nanpercentile(width.dropna(), 90))*100.0,
                f"% reenter≤{reenter_N}d after outside": pct_reenter_after_outside,
            }

            # Hit rates & conditional forward returns
            for name, mask in sigs.items():
                m = mask.fillna(False)
                idx_sig = m[m].index
                if len(idx_sig) == 0:
                    # fill NaNs for empty signals
                    row.update({f"{name}_n": 0})
                    for h in horizons:
                        row.update({
                            f"{name}_hit{h}d%": np.nan,
                            f"{name}_mean{h}d(bp)": np.nan,
                            f"{name}_med{h}d(bp)": np.nan,
                            f"{name}_avg_win{h}d(bp)": np.nan,
                            f"{name}_avg_loss{h}d(bp)": np.nan,
                        })
                    continue

                row[f"{name}_n"] = int(len(idx_sig))
                # Direction for "hit": breakout (crossU) expects +; crossL expects −; touches & mean-revert assumption
                for h in horizons:
                    fr = fwd[h].reindex(idx_sig)  # log fwd return
                    if name in ("crossU", "touchL", "mean_revert_L"):
                        # bullish expectation
                        hit = (fr > 0)
                    elif name in ("crossL", "touchU", "mean_revert_U"):
                        # bearish expectation
                        hit = (fr < 0)
                    else:
                        hit = fr > 0

                    # Calculate Average Win and Loss
                    wins = fr[hit]
                    losses = fr[~hit] # '~' is the boolean 'not' operator for pandas Series

                    avg_win_bp = (wins.mean() * 1e4) if not wins.empty else np.nan
                    avg_loss_bp = (losses.mean() * 1e4) if not losses.empty else np.nan

                    row[f"{name}_hit{h}d%"]   = float(hit.mean())
                    # report in basis points for readability
                    row[f"{name}_mean{h}d(bp)"] = float(fr.mean() * 1e4)
                    row[f"{name}_med{h}d(bp)"]  = float(fr.median() * 1e4)
                    row[f"{name}_avg_win{h}d(bp)"] = float(avg_win_bp)
                    row[f"{name}_avg_loss{h}d(bp)"] = float(avg_loss_bp)

            rows.append(row)

    out = pd.DataFrame(rows)
    # Nice ordering
    base_cols = ["index","model","obs","% inside","% touchU","% touchL","% crossU","% crossL",
                 "band_width_med%","band_width_p90%", f"% reenter≤{reenter_N}d after outside"]
    # We’ll just sort the remaining metric columns alphabetically
    metric_cols = [c for c in out.columns if c not in base_cols]
    out = out[base_cols + sorted(metric_cols)]
    return out

# ---- Run it ----
set_boll_counts()
predictions_train, predictions_test = split_data(predictions)
boll_stats_train = summarize_boll_stats(predictions_train, indexes, models, horizons=(1,3,5,10), reenter_N=5)
boll_stats_test = summarize_boll_stats(predictions_test, indexes, models, horizons=(1,3,5,10), reenter_N=5)

# Example: filter to signals with decent sample size and view
pd.set_option('display.max_columns', None)
print("--- Training Set Statistics ---")
display(boll_stats_train.sort_values(["index","model"]))

print("\n--- Test Set Statistics ---")
display(boll_stats_test.sort_values(["index","model"]))



In [ ]:
# Create backtest df copy
bt = predictions.copy()

# Set initial conditions
starting_cash = 10000
bt_data = {}

for index in indexes:
    bt_data[index] = {}

    for model in models:
        bt[f'{index}_portfolio_value_{model}'] = 0
        bt[f'{index}_cash_{model}'] = 0
        bt[f'{index}_cash_{model}'].iloc[0] = starting_cash
        bt[f'{index}_signal_{model}'] = 0

        bt_data[index][model] = {
            'last_value': 0,
            'last_cash': starting_cash,
            'hold': 0
        }

# Starts at second day
for idx, row in bt.iloc[1:].iterrows():
    for index in indexes:
        for model in models:
            last_value = bt_data[index][model]['last_value']
            last_cash = bt_data[index][model]['last_cash']

            close = row[f'{index}_Close']

            mu = np.exp(row[f'{index}_next_ret_pred_ridge'])
            vol = row[f'{index}_std_pred_{model}']
            rf = (1+row['ten_yr'])**(1/252)-1 # Daily
            investment = 0

            # Update portfolio value
            current_value = last_value * np.exp(row[f'{index}_log_ret'])

            # Calculate total equity
            total_equity = current_value + last_cash 

            # Get bet sizing with Merton
            #target_pos = get_merton_size(mu, rf, vol) * total_equity # Daily rf
            #target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity

            # Get Kelly criterion
            if bt_data[index][model]['hold'] <= 0:
                if row[f'{index}_cross_above_upper_{model}']:
                    p = boll_stats_train.loc[(boll_stats_train['index'] == index) & (boll_stats_train['model'] == model), 'crossU_hit3d%'].item()
                    q = 1 - p

                    avg_win = boll_stats_train.loc[(boll_stats_train['index'] == index) & (boll_stats_train['model'] == model), 'crossU_avg_win3d(bp)'].item()
                    avg_loss = boll_stats_train.loc[(boll_stats_train['index'] == index) & (boll_stats_train['model'] == model), 'crossU_avg_loss3d(bp)'].item()
                    
                    b = abs(avg_win / avg_loss)
                    f = min(max(0, get_kelly_criterion(b, p, q)),1)
                    target_pos = f * total_equity * .5

                    bt_data[index][model]['hold'] = 3
                elif row[f'{index}_cross_below_lower_{model}']:
                    p = boll_stats_train.loc[(boll_stats_train['index'] == index) & (boll_stats_train['model'] == model), 'crossL_hit3d%'].item()
                    q = 1 - p

                    avg_win = boll_stats_train.loc[(boll_stats_train['index'] == index) & (boll_stats_train['model'] == model), 'crossL_avg_win3d(bp)'].item()
                    avg_loss = boll_stats_train.loc[(boll_stats_train['index'] == index) & (boll_stats_train['model'] == model), 'crossL_avg_loss3d(bp)'].item()
                    
                    b = abs(avg_win / avg_loss)
                    f = -min(max(0, get_kelly_criterion(b, p, q)),1)
                    target_pos = f * total_equity * .5

                    bt_data[index][model]['hold'] = 3
                else:
                    target_pos = 0
            else:
                bt_data[index][model]['hold'] -= 1
                if bt_data[index][model]['hold'] == 0:
                    target_pos = 0


            # Get bet sizing with vol target
            #target_pos = get_vol_target_sizing(.1/np.sqrt(252), vol) * total_equity
            #target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity
            
            # Hybrid - Scale it with signed confidence from mu
            #base_weight = get_vol_target_sizing(0.1 / np.sqrt(252), vol)
            #k=1
            #confidence = (mu - rf) / (vol**2 + EPS) # Normalize mu to daily risk-free rate or historical mean
            #target_weight = base_weight * np.tanh(confidence * k)  # `k` is a scaling factor
            #target_pos = target_weight * total_equity
            #target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity

            # Bet sizing based on size of potential increase/decrease, signed
            #target_pos = (mu - rf)/abs(mu - rf) * total_equity # super leverage
            #target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity

            # Execute trade / Update investment amount to reach target position
            investment = target_pos - current_value

            # Update cash
            last_cash = bt_data[index][model]['last_cash'] = last_cash - investment
            bt.loc[idx, f'{index}_cash_{model}'] = last_cash

            # Store portfolio value as last value for calculation
            bt_data[index][model]['last_value'] = current_value + investment
            bt.loc[idx, f'{index}_portfolio_value_{model}'] = bt_data[index][model]['last_value']

            # Set signal for plotting later
            if investment > 0:
                bt.loc[idx, f'{index}_signal_{model}'] = 1
            elif investment < 0:
                bt.loc[idx, f'{index}_signal_{model}'] = -1
    
for index in indexes:
    for model in models:
        bt[f'{index}_total_value_{model}'] = bt[f'{index}_portfolio_value_{model}'] + bt[f'{index}_cash_{model}']
        final_value = bt[f'{index}_total_value_{model}'].iloc[-1]
        print(f'Final value for {index}, {model}: {final_value:.2f}')
    print('\n')

In [ ]:
plot_results(bt, 'baseline_wealth', ['total_value'], 'Backtest Performance', 'Wealth', indexes, models)

# MS-Garch Volatility Model

In [ ]:
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
from arch.univariate import ConstantMean, GARCH, StudentsT
import re

Test pulled from GPT (python doesn't have a MS-GARCH library, only separate implementations with reduced functionality)

In [ ]:
class MSGARCH:
    """
    Markov switching means & transitions via statsmodels.MarkovRegression,
    per-state GARCH(1,1) via arch; combine with Gray (1996) approximation.
    """
    def __init__(self, k=2, thresh=0.6, max_iter=1, verbose=False, dist='normal'):
        self.k = int(k)
        self.thresh = float(thresh)  # posterior cutoff for state-specific GARCH fit
        self.max_iter = int(max_iter)
        self.verbose = verbose
        self.dist = dist.lower()
        assert self.dist in ("normal", "t"), "dist must be 'normal' or 't'"
        self.nu_ = None   # will hold per-state dof if dist='t'

    def fit(self, r: pd.Series | np.ndarray):
        r = np.asarray(pd.Series(r).astype(float).dropna())
        T = len(r)
        assert T > 50, "Need enough data"

        # 1) HMM on returns with state-dependent intercept (mean) and variance
        #    (statsmodels allows switching_variance=True, which helps separate regimes)
        mod = MarkovRegression(r, k_regimes=self.k,
                               trend='c',  # intercept per regime
                               switching_variance=True)
        res = mod.fit(disp=False)
        self._mr_mod_ = mod
        self._mr_res_ = res

        # --- 1) Extract regime-specific means (intercepts) ---
        names = np.asarray(res.model.param_names)
        vals  = np.asarray(res.params)

        mask = (np.char.find(names, 'intercept') >= 0)
        if not mask.any():
            mask = (np.char.find(names, 'const') >= 0)

        intercept_names = names[mask]
        intercept_vals  = vals[mask]

        pairs = []
        for n, v in zip(intercept_names, intercept_vals):
            m = re.search(r'\[(\d+)\]', n)
            idx = int(m.group(1)) if m else 10**9
            pairs.append((idx, float(v)))
        pairs.sort(key=lambda t: t[0])
        mu = np.array([p[1] for p in pairs])  # length K

        # --- 2) Transition matrix ---
        P_left = mod.regime_transition_matrix(res.params)  # columns sum to 1
        P = P_left.T                                      # rows sum to 1, matches our code convention

        # --- 3) Smoothed posteriors ---
        gamma_obj = res.smoothed_marginal_probabilities
        if hasattr(gamma_obj, "to_numpy"):
            gamma = gamma_obj.to_numpy()
        else:
            gamma = np.asarray(gamma_obj)                     # shape (T, K)

        # 2) Fit GARCH(1,1) separately for each regime using high-probability times
        self._arch_results_ = []
        garch_params = []
        nus = []
        for k in range(self.k):
            mask = (gamma[:, k] == gamma.max(1)) & (gamma[:, k] >= self.thresh)
            idx = np.where(mask)[0]
            if len(idx) < 30:
                idx = np.argsort(gamma[:, k])[-max(30, T // self.k):]
            r_k = r[idx] - mu[k]  # center by state mean

            am = ConstantMean(r_k)
            am.volatility = GARCH(1, 0, 1)
            if self.dist == "t":
                am.distribution = StudentsT()
            res_k = am.fit(disp="off")

            self._arch_results_.append(res_k)
            v = res_k.params
            omega = float(v["omega"])
            alpha = float(v["alpha[1]"])
            beta  = float(v["beta[1]"])

            # (optional) safety clamp on persistence
            s = max(alpha + beta, 1e-6)
            if s >= 0.998:
                shrink = (s / 0.98)
                alpha /= shrink; beta /= shrink

            garch_params.append((omega, alpha, beta))

            if self.dist == "t":
                # ARCH names this 'nu'
                nu = float(v["nu"])
                # ensure nu > 2 so variance exists
                nu = max(nu, 2.01)
                nus.append(nu)

        garch_params = np.array(garch_params)
        omega = garch_params[:, 0]; alpha = garch_params[:, 1]; beta = garch_params[:, 2]
        self.omega_, self.alpha_, self.beta_ = omega, alpha, beta
        self.nu_ = np.array(nus) if self.dist == "t" else None

        # 3) Gray recursion using gamma as pi_t
        h, m_mix, h_mix = self._gray_recursion(r, mu, omega, alpha, beta, gamma)

        # (Optional) one extra iteration: re-center residuals with mixture mean and refit GARCH
        for it in range(self.max_iter):
            if self.max_iter <= 1: break
            # reselect by posterior again (same gamma) but use residuals r - mu_k
            new_params = []
            for k in range(self.k):
                mask = (gamma[:, k] == gamma.max(1)) & (gamma[:, k] >= self.thresh)
                idx = np.where(mask)[0]
                if len(idx) < 30:
                    idx = np.argsort(gamma[:, k])[-max(30, T // self.k):]
                r_k = r[idx] - mu[k]
                am = ConstantMean(r_k)
                am.volatility = GARCH(1, 0, 1)
                res_k = am.fit(disp='off')
                self._arch_results_[k] = res_k
                v = res_k.params
                omega = float(v['omega']); alpha = float(v['alpha[1]']); beta = float(v['beta[1]'])
                s = max(alpha + beta, 1e-6)
                if s >= 0.998:
                    shrink = (s / 0.98)
                    alpha /= shrink; beta /= shrink
                new_params.append((omega, alpha, beta))
            new_params = np.array(new_params)
            omega, alpha, beta = new_params[:,0], new_params[:,1], new_params[:,2]
            h, m_mix, h_mix = self._gray_recursion(r, mu, omega, alpha, beta, gamma)

        # Store
        self.mu_ = mu
        self.P_ = P
        self.gamma_ = gamma
        self.h_ = h                 # (T,K) per-state conditional variances
        self.h_mix_ = h_mix         # (T,) mixture variance
        self.m_mix_ = m_mix         # (T,) mixture mean
        self.omega_ = omega; self.alpha_ = alpha; self.beta_ = beta
        self.r_ = r
        return self

    def _gray_recursion(self, r, mu, omega, alpha, beta, pi):
        T = len(r); K = len(mu)
        h = np.zeros((T, K))
        m_mix = np.zeros(T)
        H_mix = np.zeros(T)

        # initialize with unconditional per state
        den = np.maximum(1.0 - (alpha + beta), 1e-3)
        h0 = omega / den
        h[0] = np.maximum(h0, 1e-8)

        m_mix[0] = (pi[0] @ mu)
        H_mix[0] = (pi[0] @ h[0])

        for t in range(1, T):
            m_mix[t] = pi[t-1] @ mu
            H_mix[t] = pi[t-1] @ h[t-1]
            innov2 = (r[t-1] - m_mix[t])**2
            h[t] = omega + alpha * innov2 + beta * H_mix[t]
            h[t] = np.maximum(h[t], 1e-12)

        h_mix = (pi * h).sum(1)
        return h, m_mix, h_mix

    # One-step-ahead (no new data)
    def predict_next(self):
        r_T = self.r_[-1]
        m_T = self.m_mix_[-1]
        H_T = self.h_mix_[-1]
        pi_T = self.gamma_[-1]
        mu   = self.mu_
        omega, alpha, beta = self.omega_, self.alpha_, self.beta_
        h_next = omega + alpha * (r_T - m_T)**2 + beta * H_T
        h_next = np.maximum(h_next, 1e-12)
        pi_next = pi_T @ self.P_
        mean_next = float(pi_next @ mu)
        var_next  = float(pi_next @ h_next)
        return {"pi_next": pi_next, "h_next_per_state": h_next,
                "mean_next": mean_next, "var_next": var_next}

    def _loglik_state(self, r_new, mu_k, h_k, nu_k=None):
        """
        Log-likelihood of r_new under state k with mean mu_k, variance h_k,
        using Normal or Student-t (df=nu_k) innovations.
        """
        h = max(h_k, 1e-12)
        x = r_new - mu_k
        if self.dist == "normal":
            # N(mu, h)
            return -0.5 * (math.log(2*math.pi*h) + (x*x)/h)
        else:
            # Student-t with df=nu, using *standard* t scaled so that Var = h
            # Standard t(df) with scale s has pdf: log Γ((ν+1)/2) - log(√(νπ) s Γ(ν/2))
            #                                  - (ν+1)/2 * log(1 + ((x)/s)^2 / ν)
            nu = max(float(nu_k), 2.01)
            # choose scale so that Var(X)=h -> s = sqrt(h * ν/(ν-2))
            s = math.sqrt(h * nu/(nu-2.0))
            z2 = (x/s)**2
            return (
                math.lgamma((nu+1)/2.0)
                - math.lgamma(nu/2.0)
                - 0.5*math.log(nu*math.pi) - math.log(s)
                - 0.5*(nu+1.0)*math.log(1.0 + z2/nu)
            )

    # Online filter step (fixed params)
    def filter_update(self, r_new):
        r_new = float(r_new)
        fc = self.predict_next()
        h_next = fc["h_next_per_state"]     # per-state conditional variances for the new time
        pi_pred = fc["pi_next"]
        mu = self.mu_

        # per-state log-likelihood
        loglik = np.empty(self.k)
        for k in range(self.k):
            nu_k = None if self.dist == "normal" else self.nu_[k]
            loglik[k] = self._loglik_state(r_new, mu[k], h_next[k], nu_k=nu_k)

        # stabilize
        loglik -= loglik.max()
        lik = np.exp(loglik)

        pi_post = pi_pred * lik
        pi_post = pi_post / pi_post.sum()

        # append updated series
        self.r_ = np.append(self.r_, r_new)
        self.gamma_ = np.vstack([self.gamma_, pi_post])

        mix_var = float(pi_post @ h_next)
        mix_mean = float(pi_post @ mu)
        self.h_mix_ = np.append(self.h_mix_, mix_var)
        self.m_mix_ = np.append(self.m_mix_, mix_mean)

        return {"pi_post": pi_post, "mix_mean": mix_mean, "mix_var": mix_var}


In [ ]:
ms_garch_data = vol_data.copy()

training_data, testing_data = split_data(ms_garch_data)

training_data

In [ ]:
garches = {}


for index in indexes:
    r = np.exp(training_data[f'{index}_log_squared_residual'])
    garches[index] = MSGARCH(k=3, thresh=0.8, max_iter=1, verbose=True, dist='t')
    garches[index].fit(r)

In [ ]:
# Create backtest df copy
bt = predictions.copy()

# Set initial conditions
starting_cash = 10000
bt_data = {}

# Model
model_name = 'MS_GARCH'

trade_futures = False
futures_columns = '_F' if trade_futures else ''

for index in indexes:
    bt_data[index] = {}


    bt[f'{index}_portfolio_value_{model_name}'] = 0
    bt[f'{index}_cash_{model_name}'] = 0
    bt[f'{index}_cash_{model_name}'].iloc[0] = starting_cash
    bt[f'{index}_signal_{model_name}'] = 0

    bt_data[index][model_name] = {
        'last_value': 0,
        'last_cash': starting_cash
    }

# Starts at second day
for idx, row in bt.iloc[1:].iterrows():
    for index in indexes:
        last_value = bt_data[index][model_name]['last_value']
        last_cash = bt_data[index][model_name]['last_cash']

        close = row[f'{index}_Close']
        rf = (1+row['ten_yr'])**(1/252)-1 # Daily

        div_dict = {'S&P': 'SPY', 'NASDAQ': 'QQQ', 'DJIA': 'DIA'}
        div_name = f'{div_dict[index]}_div'
        div = (1+row[div_name])**(1/252)-1 # Daily

        fc = garches[index].predict_next()
        garches[index].filter_update(row[f'{index}{futures_columns}_log_ret'])

        #mu = (fc['mean_next']/100) / close - 1 # Use garch prediction for returns
        mu = row[f'{index}_next_ret_pred_enet'] / close - 1 # Use our regression model prediction for returns
        #mu = rf - div # Use risk free assumption
        vol = np.sqrt(fc['var_next'])
        
        
        investment = 0

        # Update portfolio value
        current_value = last_value * np.exp(row[f'{index}{futures_columns}_log_ret'])

        # Calculate total equity
        total_equity = current_value + last_cash 

        # Get bet sizing with Merton-Kelly
        #target_pos = .5 * get_merton_kelly_size(mu, rf, vol) * total_equity # Daily rf
        #target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity

        # Get bet sizing with vol target
        target_pos = get_vol_target_sizing(.1/np.sqrt(252), vol) * total_equity
        target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity
        
        # Bet sizing based on size of potential increase/decrease, signed
        #target_pos = (mu - rf)/abs(mu - rf) * total_equity # super leverage
        #target_pos = max(-total_equity, min(target_pos, total_equity)) # Make sure we don't exceed equity

        # Execute trade / Update investment amount to reach target position
        investment = target_pos - current_value

        # Update cash
        last_cash = bt_data[index][model_name]['last_cash'] = last_cash - investment
        bt.loc[idx, f'{index}_cash_{model_name}'] = last_cash

        # Store portfolio value as last value for calculation
        bt_data[index][model_name]['last_value'] = current_value + investment
        bt.loc[idx, f'{index}_portfolio_value_{model_name}'] = bt_data[index][model_name]['last_value']

        # Set signal for plotting later
        if investment > 0:
            bt.loc[idx, f'{index}_signal_{model_name}'] = 1
        elif investment < 0:
            bt.loc[idx, f'{index}_signal_{model_name}'] = -1

    
for index in indexes:
    bt[f'{index}_total_value_{model_name}'] = bt[f'{index}_portfolio_value_{model_name}'] + bt[f'{index}_cash_{model_name}']
    final_value = bt[f'{index}_total_value_{model_name}'].iloc[-1]
    print(f'Final value for {index}, {model_name}: {final_value:.2f}')
    print('\n')

In [ ]:
for index in indexes:
    bt[f'{index}_baseline'] = np.exp(bt[f'{index}_log_ret'].cumsum()) * 10000

colors = { 'MS_GARCH': 'red' }

# --- Plotting Code ---

# Initialize a figure with a row for each index
fig = make_subplots(
    rows=len(indexes),
    cols=1,
    subplot_titles=[f'{index} Model Performance' for index in indexes],
    shared_xaxes=True # Link the x-axes
)

# Enumerate through indexes to get the row number (i)
for i, index in enumerate(indexes):
    # Add the baseline trace for the current index
    fig.add_trace(go.Scatter(
        x=bt.index,
        y=bt[f'{index}_baseline'],
        mode='lines',
        name=f'{index} Baseline',
        legendgroup=f'group{i}', # Group legend items by subplot
        line=dict(color='blue', width=1)
    ), row=i + 1, col=1) # row is 1-indexed

    # Add the main performance line for the model
    fig.add_trace(go.Scatter(
        x=bt.index,
        y=bt[f'{index}_total_value_{model_name}'],
        mode='lines',
        name=f'{index} {model_name}',
        legendgroup=f'group{i}',
        line=dict(color=colors[model_name], width=1.5, dash='dot')
    ), row=i + 1, col=1)

    # --- Corrected Signal Plotting ---

    # Create Series for buy/sell signals
    # This puts the portfolio value on the y-axis for the marker, and NaN otherwise
    buy_signals_y = bt.loc[bt[f'{index}_signal_{model_name}'] == 1, f'{index}_total_value_{model_name}']
    sell_signals_y = bt.loc[bt[f'{index}_signal_{model_name}'] == -1, f'{index}_total_value_{model_name}']

""" These are too crowded
        # Add BUY signal markers
        fig.add_trace(go.Scatter(
            x=buy_signals_y.index,
            y=buy_signals_y,
            mode='markers',
            name=f'Buy Signal',
            legendgroup=f'group{i}',
            marker=dict(size=10, symbol='triangle-up', color='green'),
            showlegend=False # Hide from legend to avoid clutter
        ), row=i + 1, col=1)

        # Add SELL signal markers
        fig.add_trace(go.Scatter(
            x=sell_signals_y.index,
            y=sell_signals_y,
            mode='markers',
            name=f'Sell Signal',
            legendgroup=f'group{i}',
            marker=dict(size=10, symbol='triangle-down', color='red'),
            showlegend=False # Hide from legend to avoid clutter
        ), row=i + 1, col=1)
        """

# --- Update the Layout ---
fig.update_layout(
    title_text='Backtest Performance: Model Comparison by Index',
    title_x=0.5, # Center the title
    legend_title='Metrics',
    height=800 # Adjust height to make plots more readable
)

# --- Display the Chart ---
fig.show()

cutoff = math.floor(len(bt)*.8)
training_returns = bt.iloc[:cutoff]
testing_returns = bt.iloc[cutoff:]

for index in indexes:
    total_value = f'{index}_total_value_{model_name}'
    final_training_value = training_returns[total_value].iloc[-1]
    training_cagr = 100 * ((final_training_value/training_returns[total_value].iloc[0])**(1/(training_returns.index[-1].year - training_returns.index[0].year + training_returns.index[-1].month/12))-1)
    print(f'Final training value for {index}, {model_name}: {final_training_value}, CAGR: {training_cagr:.2f}%')

    final_testing_value = testing_returns[total_value].iloc[-1]
    testing_cagr = 100*((final_testing_value/testing_returns[total_value].iloc[0])**(1/(testing_returns.index[-1].year - testing_returns.index[0].year + testing_returns.index[-1].month/12))-1)
    print(f'Final testing value for {index}, {model_name}: {final_testing_value}, CAGR: {testing_cagr:.2f}%')

    # ---- Training metrics ----
    train_curve = training_returns[total_value]
    train_sharpe, train_mdd = sharpe_and_mdd(training_returns, total_value, col_rf="ten_yr")
    print(f"Training Sharpe: {train_sharpe:.2f}, Max Drawdown: {train_mdd:.2%}")

    # ---- Testing metrics ----
    test_curve = testing_returns[total_value]
    test_sharpe, test_mdd = sharpe_and_mdd(testing_returns, total_value, col_rf="ten_yr")
    print(f"Testing Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}")

    print('\n')
